In [1]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [2]:
import xgboost as xgb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
import os
import glob
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV

## xgboost （先頭30フレームの平均）

In [3]:
# 処理するフォルダのパス
folder_path = './output_filtered_questions'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `answer1`から`answer10`のみをフィルタリング
    filtered_df = df[df["label"].str.match('^answer(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"answer{i}" for i in range(1, 11)],
        ordered=True
    )
    
    first_30_frames_per_label = (
        filtered_df.groupby("label", group_keys=False)  # 修正箇所
        .apply(lambda group: group.head(30))
        .reset_index(drop=True)
    )

    
    # 平均を計算
    mean_values_per_label = first_30_frames_per_label.groupby("label").mean(numeric_only=True)
    
    # NaNを含む行を削除
    mean_values_per_label = mean_values_per_label.dropna(axis=0)
    
    # Yを追加
    mean_values_per_label["Y"] = labels[i]
    i = i + 1
    
    # 結果をリストに保存
    result_list.append((file_name, mean_values_per_label))

# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30.csv:
          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer1     0.996101  0.621464  0.302809  0.478347  0.316149  0.184034   
answer2     0.997114  0.621363  0.350285  0.505225  0.273179  0.185705   
answer3     0.995380  0.640608  0.414539  0.464368  0.397188  0.157348   
answer4     0.994671  0.688352  0.413224  0.430911  0.422744  0.150436   
answer5     0.994515  0.686247  0.407986  0.458798  0.447200  0.149847   
answer6     0.995428  0.689668  0.524745  0.374945  0.436807  0.159959   
answer7     0.994965  0.625367  0.417123  0.453035  0.481933  0.189774   
answer8     0.994378  0.681734  0.408478  0.559817  0.323617  0.161812   
answer9     0.996479  0.583961  0.364557  0.477847  0.410745  0.205643   
answer10    0.994109  0.693208  0.466591  0.512081  0.494155  0.179663   

              AU07      AU09      AU10      AU11  ...      AU15      AU17  \
lab

          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer2     0.999810  0.470337  0.384275  0.497060  0.249186  0.462296   
answer3     0.999534  0.578685  0.501479  0.350027  0.321227  0.234205   
answer4     0.999725  0.462028  0.397030  0.600158  0.259536  0.439840   
answer6     0.999292  0.692105  0.566524  0.403788  0.344003  0.210392   
answer7     0.999482  0.690867  0.566878  0.352783  0.393997  0.236357   
answer8     0.999795  0.590483  0.471946  0.461807  0.270953  0.386812   
answer9     0.999388  0.774003  0.618143  0.398584  0.563202  0.102834   
answer10    0.999230  0.759711  0.570427  0.436846  0.562095  0.108116   

              AU07      AU09      AU10      AU11  ...      AU15      AU17  \
label                                             ...                       
answer2   0.566667  0.563768  0.471374  0.500000  ...  0.462641  0.428989   
answer3   0.333333  0.391284

          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer1     0.999485  0.782956  0.541831  0.366699  0.426326  0.186288   
answer2     0.999778  0.731527  0.465477  0.236685  0.377282  0.174415   
answer3     0.999837  0.733226  0.493464  0.333827  0.401523  0.187673   
answer4     0.999532  0.742945  0.515208  0.256200  0.340037  0.216274   
answer5     0.999805  0.776707  0.468054  0.416226  0.282310  0.287725   
answer6     0.999754  0.577312  0.479599  0.330994  0.364635  0.107069   
answer7     0.999833  0.734193  0.477516  0.276860  0.308133  0.248725   
answer8     0.999813  0.858366  0.593132  0.562592  0.235238  0.551315   
answer9     0.997955  0.591749  0.391886  0.124392  0.386063  0.296820   
answer10    0.999706  0.778841  0.459174  0.513627  0.387470  0.194525   

              AU07      AU09      AU10      AU11  ...      AU15      AU17  \
label                             

          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer2     0.998665  0.814489  0.684723  0.199340  0.388421  0.118679   
answer3     0.998857  0.799718  0.687791  0.304394  0.355658  0.118219   
answer4     0.998962  0.785451  0.677059  0.291654  0.333209  0.117223   
answer5     0.999069  0.732227  0.680503  0.150898  0.304365  0.171980   
answer7     0.998457  0.662383  0.607139  0.298247  0.307241  0.122799   
answer10    0.999170  0.703891  0.490531  0.169775  0.387965  0.156676   

              AU07      AU09      AU10      AU11  ...      AU15      AU17  \
label                                             ...                       
answer2   0.035714  0.143075  0.014057  0.607143  ...  0.434714  0.506610   
answer3   0.100000  0.161255  0.005933  0.500000  ...  0.419627  0.507973   
answer4   0.066667  0.160509  0.006057  0.600000  ...  0.434243  0.537574   
answer5   0.166667  0.

          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer1     0.991169  0.474078  0.284665  0.658233  0.419745  0.088143   
answer2     0.955926  0.623366  0.597317  0.335113  0.557142  0.166494   
answer3     0.973877  0.569425  0.488061  0.370225  0.455284  0.261796   
answer4     0.964466  0.589731  0.527762  0.378252  0.573213  0.212940   
answer5     0.937843  0.538225  0.508288  0.316699  0.489007  0.229136   
answer6     0.938234  0.597254  0.574738  0.343853  0.395508  0.334359   
answer7     0.953407  0.609050  0.488470  0.313702  0.454050  0.430063   
answer8     0.931669  0.584867  0.521922  0.347181  0.481296  0.223545   
answer9     0.955380  0.637211  0.569030  0.385576  0.674388  0.106529   
answer10    0.945165  0.592062  0.536806  0.288254  0.482736  0.221796   

              AU07      AU09      AU10      AU11  ...      AU15      AU17  \
label                             

          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer1     0.996463  0.401102  0.184681  0.736853  0.291064  0.114244   
answer2     0.998654  0.375778  0.283782  0.716622  0.391533  0.074387   
answer3     0.998215  0.521979  0.427677  0.573603  0.386193  0.125431   
answer4     0.997661  0.606746  0.452088  0.469427  0.346572  0.082818   
answer5     0.997693  0.448704  0.388612  0.465874  0.405388  0.112664   
answer6     0.997671  0.393918  0.311564  0.498517  0.385713  0.122416   
answer7     0.998256  0.467197  0.317440  0.556995  0.306403  0.093419   
answer8     0.997640  0.468572  0.359297  0.460598  0.308777  0.097490   
answer9     0.997832  0.415259  0.328217  0.575701  0.333292  0.082504   
answer10    0.997874  0.423762  0.321039  0.491157  0.330267  0.105660   

              AU07      AU09      AU10      AU11  ...      AU15      AU17  \
label                             

          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer2     0.981929  0.443911  0.323215  0.531163  0.377175  0.575861   
answer5     0.962098  0.560495  0.451856  0.438408  0.389931  0.356795   
answer6     0.990261  0.488704  0.295264  0.600758  0.350223  0.280434   
answer7     0.983672  0.493211  0.291552  0.724090  0.335082  0.333755   
answer8     0.973729  0.547745  0.485843  0.546355  0.427511  0.347061   
answer9     0.983883  0.523270  0.405973  0.633928  0.355669  0.313100   
answer10    0.983606  0.546302  0.413798  0.597647  0.384332  0.354224   

              AU07      AU09      AU10      AU11  ...      AU15      AU17  \
label                                             ...                       
answer2   0.600000  0.495324  0.566325  0.666667  ...  0.407221  0.421268   
answer5   0.733333  0.428473  0.336266  0.433333  ...  0.384424  0.513987   
answer6   0.466667  0.459

In [5]:
# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []
correct_labels_list = []  # 正解ラベルを保存するリスト

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # XGBoostモデルの初期化と訓練
    model = xgb.XGBClassifier(eval_metric='logloss')
    model.fit(X_train_combined, y_train_combined)
    
    # 予測
    y_pred = model.predict(X_test)
    
    # 精度計算
    correct_predictions = (y_pred == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100  # 予測率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    correct_labels_list.append(y_test)  # 正解ラベルも保存
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.iloc[0]}")  # 最初の正解ラベルのみ表示
    print(f"予測ラベル: {y_pred}")  # 予測ラベル全体を表示

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = 0
for (file_name, acc), y_test in zip(accuracy_list, correct_labels_list):
    if 1 in y_test.values and acc >= 50:  # 正解ラベルが1を含んでいる場合
        count_above_50_for_label_1 += 1

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")



===== テストデータ: ファイル 1/31 =====
ファイル名: filtered_ID30.csv
予測率: 100.00%
正解ラベル: 0
予測ラベル: [0 0 0 0 0 0 0 0 0 0]

===== テストデータ: ファイル 2/31 =====
ファイル名: filtered_ID31.csv
予測率: 80.00%
正解ラベル: 0
予測ラベル: [0 0 0 1 0 0 1 0 0 0]

===== テストデータ: ファイル 3/31 =====
ファイル名: filtered_ID32.csv
予測率: 20.00%
正解ラベル: 1
予測ラベル: [0 0 0 1 1 0 0 0 0 0]

===== テストデータ: ファイル 4/31 =====
ファイル名: filtered_ID33.csv
予測率: 100.00%
正解ラベル: 0
予測ラベル: [0 0 0 0 0 0 0 0 0 0]

===== テストデータ: ファイル 5/31 =====
ファイル名: filtered_ID34.csv
予測率: 62.50%
正解ラベル: 1
予測ラベル: [1 1 1 1 1 0 0 0]

===== テストデータ: ファイル 6/31 =====
ファイル名: filtered_ID35.csv
予測率: 33.33%
正解ラベル: 0
予測ラベル: [0 1 1 1 1 0 1 1 0]

===== テストデータ: ファイル 7/31 =====
ファイル名: filtered_ID36.csv
予測率: 30.00%
正解ラベル: 0
予測ラベル: [1 0 1 1 1 0 1 0 1 1]

===== テストデータ: ファイル 8/31 =====
ファイル名: filtered_ID37.csv
予測率: 70.00%
正解ラベル: 0
予測ラベル: [0 0 1 1 0 0 0 0 1 0]

===== テストデータ: ファイル 9/31 =====
ファイル名: filtered_ID38.csv
予測率: 100.00%
正解ラベル: 0
予測ラベル: [0 0 0 0 0 0 0 0 0 0]

===== テストデータ: ファイル 10/31 =====
ファイル名: filtered_I

In [4]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []
correct_labels_list = []  # 正解ラベルを保存するリスト

# グリッドサーチ用のパラメータ設定
param_grid = {
    'max_depth': [3, 5, 7],           # 木の最大深さ
    'learning_rate': [0.1, 0.01],     # 学習率
    'n_estimators': [50, 100, 200],   # 木の数
    'subsample': [0.8, 1.0],          # サブサンプリング比率
    'colsample_bytree': [0.8, 1.0],   # 特徴量サンプリング比率
}

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    test_df_cleaned = test_df_cleaned.dropna(axis=0)  # NaNを含む行を削除
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            train_df_cleaned = train_df_cleaned.dropna(axis=0)  # NaNを含む行を削除
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # XGBoostモデルをグリッドサーチでチューニング
    xgb_model = xgb.XGBClassifier(eval_metric='logloss')
    grid_search = GridSearchCV(
        estimator=xgb_model,
        param_grid=param_grid,
        scoring='accuracy',   # 評価指標は正答率
        cv=3,                 # 3分割交差検証
        n_jobs=-1,            # 並列処理
        verbose=1             # 進捗状況を表示
    )
    
    # グリッドサーチ実行
    grid_search.fit(X_train_combined, y_train_combined)
    best_model = grid_search.best_estimator_  # 最良モデルを取得
    
    # 予測
    y_pred = best_model.predict(X_test)
    
    # 精度計算
    correct_predictions = (y_pred == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100  # 予測率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    correct_labels_list.append(y_test)  # 正解ラベルも保存
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"最適パラメータ: {grid_search.best_params_}")
    print(f"正解ラベル: {y_test.iloc[0]}")
    print(f"予測ラベル: {y_pred}")

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = 0
for (file_name, acc), y_test in zip(accuracy_list, correct_labels_list):
    if 1 in y_test.values and acc >= 50:  # 正解ラベルが1を含んでいる場合
        count_above_50_for_label_1 += 1

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
Fitting 3 folds for each of 72 candidates, totalling 216 fits
ファイル名: filtered_ID30.csv
予測率: 100.00%
最適パラメータ: {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 50, 'subsample': 0.8}
正解ラベル: 0
予測ラベル: [0 0 0 0 0 0 0 0 0 0]

===== テストデータ: ファイル 2/31 =====
Fitting 3 folds for each of 72 candidates, totalling 216 fits
ファイル名: filtered_ID31.csv
予測率: 70.00%
最適パラメータ: {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 50, 'subsample': 0.8}
正解ラベル: 0
予測ラベル: [0 1 0 1 0 0 1 0 0 0]

===== テストデータ: ファイル 3/31 =====
Fitting 3 folds for each of 72 candidates, totalling 216 fits
ファイル名: filtered_ID32.csv
予測率: 10.00%
最適パラメータ: {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200, 'subsample': 0.8}
正解ラベル: 1
予測ラベル: [0 0 0 0 1 0 0 0 0 0]

===== テストデータ: ファイル 4/31 =====
Fitting 3 folds for each of 72 candidates, totalling 216 fits
ファイル名: filtered_ID33.csv
予測率: 100.00%
最適パラメータ: {'colsample_bytree': 

ファイル名: filtered_ID73.csv
予測率: 100.00%
最適パラメータ: {'colsample_bytree': 1.0, 'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 50, 'subsample': 0.8}
正解ラベル: 0
予測ラベル: [0 0 0 0 0 0 0 0 0]

===== 全ファイルごとの予測率 =====
ファイル名: filtered_ID30.csv, 予測率: 100.00%
ファイル名: filtered_ID31.csv, 予測率: 70.00%
ファイル名: filtered_ID32.csv, 予測率: 10.00%
ファイル名: filtered_ID33.csv, 予測率: 100.00%
ファイル名: filtered_ID34.csv, 予測率: 50.00%
ファイル名: filtered_ID35.csv, 予測率: 33.33%
ファイル名: filtered_ID36.csv, 予測率: 30.00%
ファイル名: filtered_ID37.csv, 予測率: 90.00%
ファイル名: filtered_ID38.csv, 予測率: 100.00%
ファイル名: filtered_ID39.csv, 予測率: 80.00%
ファイル名: filtered_ID40.csv, 予測率: 90.00%
ファイル名: filtered_ID41.csv, 予測率: 100.00%
ファイル名: filtered_ID42.csv, 予測率: 100.00%
ファイル名: filtered_ID43.csv, 予測率: 0.00%
ファイル名: filtered_ID44.csv, 予測率: 0.00%
ファイル名: filtered_ID45.csv, 予測率: 75.00%
ファイル名: filtered_ID46.csv, 予測率: 0.00%
ファイル名: filtered_ID47.csv, 予測率: 22.22%
ファイル名: filtered_ID48.csv, 予測率: 40.00%
ファイル名: filtered_ID49.csv, 予測率: 66.67%
ファイル名: filtered_ID50.csv, 予

## xgboost（先頭10フレームの平均）

In [5]:
# 処理するフォルダのパス
folder_path = './output_filtered_questions'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `answer1`から`answer10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^answer(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"answer{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の10フレームを取得
    first_10_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(10))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )
    
    # 平均を計算
    mean_values_per_label = first_10_frames_per_label.groupby("label").mean(numeric_only=True)
    
    # NaNを含む行を削除
    mean_values_per_label = mean_values_per_label.dropna(axis=0)
    
    # Yを追加
    mean_values_per_label["Y"] = labels[i]
    i = i + 1
    
    # 結果をリストに保存
    result_list.append((file_name, mean_values_per_label))

# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30.csv:
          Confidence      AU01      AU02      AU04      AU05      AU06  AU07  \
label                                                                          
answer1     0.996606  0.600396  0.295082  0.447557  0.334181  0.174490   0.1   
answer2     0.997146  0.575426  0.358429  0.525853  0.273889  0.171827   0.1   
answer3     0.996037  0.578892  0.316704  0.497932  0.299455  0.160776   0.1   
answer4     0.994471  0.672254  0.385879  0.482804  0.390718  0.150700   0.1   
answer5     0.994645  0.637202  0.329364  0.515847  0.436510  0.156992   0.1   
answer6     0.995012  0.647892  0.478994  0.362251  0.394591  0.158138   0.0   
answer7     0.995345  0.620362  0.375833  0.499724  0.418321  0.203986   0.3   
answer8     0.994680  0.668023  0.363513  0.528893  0.358564  0.158290   0.1   
answer9     0.995906  0.614710  0.362841  0.470365  0.422107  0.167508   0.0   
answer10    0.993830  0.698336  0.482286  0.404548  0.504191  0.169933   0.2   

        

          Confidence      AU01      AU02      AU04      AU05      AU06  AU07  \
label                                                                          
answer1     0.953352  0.555111  0.560406  0.338747  0.419638  0.393111   0.4   
answer2     0.954127  0.646361  0.572048  0.309115  0.373852  0.275138   0.4   
answer3     0.972704  0.667704  0.627981  0.259436  0.409858  0.234959   0.3   
answer4     0.974702  0.507010  0.421410  0.334856  0.371126  0.278374   0.4   
answer5     0.962499  0.648968  0.578337  0.291137  0.460339  0.377172   0.6   
answer6     0.975186  0.675066  0.703820  0.239825  0.500465  0.168293   0.3   
answer8     0.969312  0.685026  0.569289  0.331068  0.579876  0.196200   0.3   
answer9     0.985019  0.626181  0.565170  0.232544  0.431355  0.269582   0.6   
answer10    0.984534  0.555437  0.475906  0.291157  0.406598  0.372321   0.4   

              AU09      AU10  AU11  ...      AU15      AU17  AU20      AU23  \
label                               ... 

In [11]:
# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []
correct_labels_list = []  # 正解ラベルを保存するリスト

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # XGBoostモデルの初期化と訓練
    model = xgb.XGBClassifier(eval_metric='logloss')
    model.fit(X_train_combined, y_train_combined)
    
    # 予測
    y_pred = model.predict(X_test)
    
    # 精度計算
    correct_predictions = (y_pred == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100  # 予測率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    correct_labels_list.append(y_test)  # 正解ラベルも保存
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.iloc[0]}")  # 最初の正解ラベルのみ表示
    print(f"予測ラベル: {y_pred}")  # 予測ラベル全体を表示

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = 0
for (file_name, acc), y_test in zip(accuracy_list, correct_labels_list):
    if 1 in y_test.values and acc >= 50:  # 正解ラベルが1を含んでいる場合
        count_above_50_for_label_1 += 1

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
ファイル名: filtered_ID30.csv
予測率: 100.00%
正解ラベル: 0
予測ラベル: [0 0 0 0 0 0 0 0 0 0]

===== テストデータ: ファイル 2/31 =====
ファイル名: filtered_ID31.csv
予測率: 90.00%
正解ラベル: 0
予測ラベル: [0 1 0 0 0 0 0 0 0 0]

===== テストデータ: ファイル 3/31 =====
ファイル名: filtered_ID32.csv
予測率: 10.00%
正解ラベル: 1
予測ラベル: [0 0 0 0 1 0 0 0 0 0]

===== テストデータ: ファイル 4/31 =====
ファイル名: filtered_ID33.csv
予測率: 100.00%
正解ラベル: 0
予測ラベル: [0 0 0 0 0 0 0 0 0 0]

===== テストデータ: ファイル 5/31 =====
ファイル名: filtered_ID34.csv
予測率: 62.50%
正解ラベル: 1
予測ラベル: [1 1 0 1 1 0 1 0]

===== テストデータ: ファイル 6/31 =====
ファイル名: filtered_ID35.csv
予測率: 77.78%
正解ラベル: 0
予測ラベル: [0 0 0 0 1 0 0 1 0]

===== テストデータ: ファイル 7/31 =====
ファイル名: filtered_ID36.csv
予測率: 40.00%
正解ラベル: 0
予測ラベル: [0 0 1 1 1 1 1 0 0 1]

===== テストデータ: ファイル 8/31 =====
ファイル名: filtered_ID37.csv
予測率: 80.00%
正解ラベル: 0
予測ラベル: [0 0 1 0 0 0 0 0 0 1]

===== テストデータ: ファイル 9/31 =====
ファイル名: filtered_ID38.csv
予測率: 100.00%
正解ラベル: 0
予測ラベル: [0 0 0 0 0 0 0 0 0 0]

===== テストデータ: ファイル 10/31 =====
ファイル名: filtered_I

In [6]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []
correct_labels_list = []  # 正解ラベルを保存するリスト

# グリッドサーチ用のパラメータ設定
param_grid = {
    'max_depth': [3, 5, 7],           # 木の最大深さ
    'learning_rate': [0.1, 0.01],     # 学習率
    'n_estimators': [50, 100, 200],   # 木の数
    'subsample': [0.8, 1.0],          # サブサンプリング比率
    'colsample_bytree': [0.8, 1.0],   # 特徴量サンプリング比率
}

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    test_df_cleaned = test_df_cleaned.dropna(axis=0)  # NaNを含む行を削除
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            train_df_cleaned = train_df_cleaned.dropna(axis=0)  # NaNを含む行を削除
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # XGBoostモデルをグリッドサーチでチューニング
    xgb_model = xgb.XGBClassifier(eval_metric='logloss')
    grid_search = GridSearchCV(
        estimator=xgb_model,
        param_grid=param_grid,
        scoring='accuracy',   # 評価指標は正答率
        cv=3,                 # 3分割交差検証
        n_jobs=-1,            # 並列処理
        verbose=1             # 進捗状況を表示
    )
    
    # グリッドサーチ実行
    grid_search.fit(X_train_combined, y_train_combined)
    best_model = grid_search.best_estimator_  # 最良モデルを取得
    
    # 予測
    y_pred = best_model.predict(X_test)
    
    # 精度計算
    correct_predictions = (y_pred == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100  # 予測率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    correct_labels_list.append(y_test)  # 正解ラベルも保存
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"最適パラメータ: {grid_search.best_params_}")
    print(f"正解ラベル: {y_test.iloc[0]}")
    print(f"予測ラベル: {y_pred}")

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = 0
for (file_name, acc), y_test in zip(accuracy_list, correct_labels_list):
    if 1 in y_test.values and acc >= 50:  # 正解ラベルが1を含んでいる場合
        count_above_50_for_label_1 += 1

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
Fitting 3 folds for each of 72 candidates, totalling 216 fits
ファイル名: filtered_ID30.csv
予測率: 100.00%
最適パラメータ: {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 50, 'subsample': 0.8}
正解ラベル: 0
予測ラベル: [0 0 0 0 0 0 0 0 0 0]

===== テストデータ: ファイル 2/31 =====
Fitting 3 folds for each of 72 candidates, totalling 216 fits
ファイル名: filtered_ID31.csv
予測率: 80.00%
最適パラメータ: {'colsample_bytree': 1.0, 'learning_rate': 0.01, 'max_depth': 7, 'n_estimators': 50, 'subsample': 1.0}
正解ラベル: 0
予測ラベル: [0 1 0 0 0 0 0 0 0 1]

===== テストデータ: ファイル 3/31 =====
Fitting 3 folds for each of 72 candidates, totalling 216 fits
ファイル名: filtered_ID32.csv
予測率: 10.00%
最適パラメータ: {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 200, 'subsample': 0.8}
正解ラベル: 1
予測ラベル: [0 0 0 0 1 0 0 0 0 0]

===== テストデータ: ファイル 4/31 =====
Fitting 3 folds for each of 72 candidates, totalling 216 fits
ファイル名: filtered_ID33.csv
予測率: 100.00%
最適パラメータ: {'colsample_bytree'

ファイル名: filtered_ID73.csv
予測率: 100.00%
最適パラメータ: {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 50, 'subsample': 0.8}
正解ラベル: 0
予測ラベル: [0 0 0 0 0 0 0 0 0]

===== 全ファイルごとの予測率 =====
ファイル名: filtered_ID30.csv, 予測率: 100.00%
ファイル名: filtered_ID31.csv, 予測率: 80.00%
ファイル名: filtered_ID32.csv, 予測率: 10.00%
ファイル名: filtered_ID33.csv, 予測率: 100.00%
ファイル名: filtered_ID34.csv, 予測率: 50.00%
ファイル名: filtered_ID35.csv, 予測率: 88.89%
ファイル名: filtered_ID36.csv, 予測率: 50.00%
ファイル名: filtered_ID37.csv, 予測率: 100.00%
ファイル名: filtered_ID38.csv, 予測率: 100.00%
ファイル名: filtered_ID39.csv, 予測率: 100.00%
ファイル名: filtered_ID40.csv, 予測率: 90.00%
ファイル名: filtered_ID41.csv, 予測率: 90.00%
ファイル名: filtered_ID42.csv, 予測率: 100.00%
ファイル名: filtered_ID43.csv, 予測率: 0.00%
ファイル名: filtered_ID44.csv, 予測率: 50.00%
ファイル名: filtered_ID45.csv, 予測率: 87.50%
ファイル名: filtered_ID46.csv, 予測率: 20.00%
ファイル名: filtered_ID47.csv, 予測率: 22.22%
ファイル名: filtered_ID48.csv, 予測率: 10.00%
ファイル名: filtered_ID49.csv, 予測率: 66.67%
ファイル名: filtered_ID50.csv

## xgboost（先頭30フレームの標準偏差）

In [7]:
# 処理するフォルダのパス
folder_path = './output_filtered_questions'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `answer1`から`answer10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^answer(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"answer{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の30フレームを取得
    first_30_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(30))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )
    
    # 標準偏差を計算
    std_values_per_label = first_30_frames_per_label.groupby("label").std(numeric_only=True)

    # NaNを含む行を削除
    std_values_per_label = std_values_per_label.dropna(axis=0)
    
    # Yを追加
    std_values_per_label["Y"] = labels[i]
    i = i + 1
    
    # 結果をリストに保存
    result_list.append((file_name, std_values_per_label))

# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30.csv:
          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer1     0.001530  0.066801  0.085971  0.122109  0.076855  0.046814   
answer2     0.000945  0.085073  0.084465  0.104862  0.052543  0.041350   
answer3     0.001572  0.103785  0.125221  0.102060  0.104269  0.032001   
answer4     0.001297  0.072478  0.086068  0.102976  0.114848  0.034969   
answer5     0.001613  0.064449  0.107979  0.117039  0.100585  0.041976   
answer6     0.001492  0.061847  0.100756  0.110784  0.105393  0.027268   
answer7     0.001258  0.062818  0.113542  0.141535  0.163639  0.037189   
answer8     0.001798  0.068074  0.090012  0.107492  0.064596  0.035957   
answer9     0.001176  0.102274  0.099719  0.121681  0.061962  0.044371   
answer10    0.001767  0.060623  0.082212  0.140823  0.071314  0.024458   

              AU07      AU09      AU10      AU11  ...      AU15      AU17  \
lab

          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer1     0.001278  0.160914  0.086505  0.140431  0.070324  0.105877   
answer2     0.002824  0.168720  0.234726  0.205987  0.090208  0.080281   
answer3     0.003218  0.106912  0.188110  0.230762  0.127930  0.043148   
answer4     0.005926  0.118801  0.186076  0.180543  0.092645  0.049175   
answer5     0.003032  0.106469  0.172500  0.183614  0.112806  0.056420   
answer6     0.003835  0.128791  0.184218  0.151517  0.080262  0.077459   
answer7     0.002169  0.097065  0.173160  0.180075  0.084040  0.153861   
answer8     0.002700  0.103597  0.173947  0.159342  0.092793  0.096565   
answer9     0.003261  0.127420  0.186532  0.186029  0.084044  0.070091   
answer10    0.001270  0.047784  0.099021  0.124619  0.065951  0.063548   

              AU07      AU09      AU10      AU11  ...      AU15      AU17  \
label                             

In [13]:
# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []
correct_labels_list = []  # 正解ラベルを保存するリスト

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # XGBoostモデルの初期化と訓練
    model = xgb.XGBClassifier(eval_metric='logloss')
    model.fit(X_train_combined, y_train_combined)
    
    # 予測
    y_pred = model.predict(X_test)
    
    # 精度計算
    correct_predictions = (y_pred == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100  # 予測率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    correct_labels_list.append(y_test)  # 正解ラベルも保存
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.iloc[0]}")  # 最初の正解ラベルのみ表示
    print(f"予測ラベル: {y_pred}")  # 予測ラベル全体を表示

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = 0
for (file_name, acc), y_test in zip(accuracy_list, correct_labels_list):
    if 1 in y_test.values and acc >= 50:  # 正解ラベルが1を含んでいる場合
        count_above_50_for_label_1 += 1

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
ファイル名: filtered_ID30.csv
予測率: 90.00%
正解ラベル: 0
予測ラベル: [0 0 0 0 1 0 0 0 0 0]

===== テストデータ: ファイル 2/31 =====
ファイル名: filtered_ID31.csv
予測率: 80.00%
正解ラベル: 0
予測ラベル: [0 0 1 1 0 0 0 0 0 0]

===== テストデータ: ファイル 3/31 =====
ファイル名: filtered_ID32.csv
予測率: 10.00%
正解ラベル: 1
予測ラベル: [0 0 0 0 0 1 0 0 0 0]

===== テストデータ: ファイル 4/31 =====
ファイル名: filtered_ID33.csv
予測率: 70.00%
正解ラベル: 0
予測ラベル: [1 0 1 0 1 0 0 0 0 0]

===== テストデータ: ファイル 5/31 =====
ファイル名: filtered_ID34.csv
予測率: 12.50%
正解ラベル: 1
予測ラベル: [0 0 0 1 0 0 0 0]

===== テストデータ: ファイル 6/31 =====
ファイル名: filtered_ID35.csv
予測率: 100.00%
正解ラベル: 0
予測ラベル: [0 0 0 0 0 0 0 0 0]

===== テストデータ: ファイル 7/31 =====
ファイル名: filtered_ID36.csv
予測率: 70.00%
正解ラベル: 0
予測ラベル: [0 0 0 1 1 0 0 0 1 0]

===== テストデータ: ファイル 8/31 =====
ファイル名: filtered_ID37.csv
予測率: 70.00%
正解ラベル: 0
予測ラベル: [0 0 1 0 1 0 0 0 0 1]

===== テストデータ: ファイル 9/31 =====
ファイル名: filtered_ID38.csv
予測率: 80.00%
正解ラベル: 0
予測ラベル: [0 0 0 0 0 0 1 0 0 1]

===== テストデータ: ファイル 10/31 =====
ファイル名: filtered_ID3

In [8]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []
correct_labels_list = []  # 正解ラベルを保存するリスト

# グリッドサーチ用のパラメータ設定
param_grid = {
    'max_depth': [3, 5, 7],           # 木の最大深さ
    'learning_rate': [0.1, 0.01],     # 学習率
    'n_estimators': [50, 100, 200],   # 木の数
    'subsample': [0.8, 1.0],          # サブサンプリング比率
    'colsample_bytree': [0.8, 1.0],   # 特徴量サンプリング比率
}

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    test_df_cleaned = test_df_cleaned.dropna(axis=0)  # NaNを含む行を削除
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            train_df_cleaned = train_df_cleaned.dropna(axis=0)  # NaNを含む行を削除
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # XGBoostモデルをグリッドサーチでチューニング
    xgb_model = xgb.XGBClassifier(eval_metric='logloss')
    grid_search = GridSearchCV(
        estimator=xgb_model,
        param_grid=param_grid,
        scoring='accuracy',   # 評価指標は正答率
        cv=3,                 # 3分割交差検証
        n_jobs=-1,            # 並列処理
        verbose=1             # 進捗状況を表示
    )
    
    # グリッドサーチ実行
    grid_search.fit(X_train_combined, y_train_combined)
    best_model = grid_search.best_estimator_  # 最良モデルを取得
    
    # 予測
    y_pred = best_model.predict(X_test)
    
    # 精度計算
    correct_predictions = (y_pred == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100  # 予測率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    correct_labels_list.append(y_test)  # 正解ラベルも保存
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"最適パラメータ: {grid_search.best_params_}")
    print(f"正解ラベル: {y_test.iloc[0]}")
    print(f"予測ラベル: {y_pred}")

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = 0
for (file_name, acc), y_test in zip(accuracy_list, correct_labels_list):
    if 1 in y_test.values and acc >= 50:  # 正解ラベルが1を含んでいる場合
        count_above_50_for_label_1 += 1

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
Fitting 3 folds for each of 72 candidates, totalling 216 fits
ファイル名: filtered_ID30.csv
予測率: 100.00%
最適パラメータ: {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 7, 'n_estimators': 50, 'subsample': 0.8}
正解ラベル: 0
予測ラベル: [0 0 0 0 0 0 0 0 0 0]

===== テストデータ: ファイル 2/31 =====
Fitting 3 folds for each of 72 candidates, totalling 216 fits
ファイル名: filtered_ID31.csv
予測率: 100.00%
最適パラメータ: {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 7, 'n_estimators': 50, 'subsample': 0.8}
正解ラベル: 0
予測ラベル: [0 0 0 0 0 0 0 0 0 0]

===== テストデータ: ファイル 3/31 =====
Fitting 3 folds for each of 72 candidates, totalling 216 fits
ファイル名: filtered_ID32.csv
予測率: 30.00%
最適パラメータ: {'colsample_bytree': 1.0, 'learning_rate': 0.01, 'max_depth': 7, 'n_estimators': 100, 'subsample': 1.0}
正解ラベル: 1
予測ラベル: [0 0 0 0 1 1 1 0 0 0]

===== テストデータ: ファイル 4/31 =====
Fitting 3 folds for each of 72 candidates, totalling 216 fits
ファイル名: filtered_ID33.csv
予測率: 90.00%
最適パラメータ: {'colsample_bytree

ファイル名: filtered_ID73.csv
予測率: 88.89%
最適パラメータ: {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 7, 'n_estimators': 50, 'subsample': 1.0}
正解ラベル: 0
予測ラベル: [0 0 0 0 0 0 0 0 1]

===== 全ファイルごとの予測率 =====
ファイル名: filtered_ID30.csv, 予測率: 100.00%
ファイル名: filtered_ID31.csv, 予測率: 100.00%
ファイル名: filtered_ID32.csv, 予測率: 30.00%
ファイル名: filtered_ID33.csv, 予測率: 90.00%
ファイル名: filtered_ID34.csv, 予測率: 12.50%
ファイル名: filtered_ID35.csv, 予測率: 88.89%
ファイル名: filtered_ID36.csv, 予測率: 70.00%
ファイル名: filtered_ID37.csv, 予測率: 100.00%
ファイル名: filtered_ID38.csv, 予測率: 100.00%
ファイル名: filtered_ID39.csv, 予測率: 90.00%
ファイル名: filtered_ID40.csv, 予測率: 100.00%
ファイル名: filtered_ID41.csv, 予測率: 100.00%
ファイル名: filtered_ID42.csv, 予測率: 88.89%
ファイル名: filtered_ID43.csv, 予測率: 0.00%
ファイル名: filtered_ID44.csv, 予測率: 0.00%
ファイル名: filtered_ID45.csv, 予測率: 100.00%
ファイル名: filtered_ID46.csv, 予測率: 10.00%
ファイル名: filtered_ID47.csv, 予測率: 55.56%
ファイル名: filtered_ID48.csv, 予測率: 40.00%
ファイル名: filtered_ID49.csv, 予測率: 83.33%
ファイル名: filtered_ID50.csv,

## xgboost（先頭10フレームの標準偏差）

In [3]:
# 処理するフォルダのパス
folder_path = './output_filtered_questions'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `answer1`から`answer10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^answer(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"answer{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の30フレームを取得
    first_30_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(10))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )
    
    # 標準偏差を計算
    std_values_per_label = first_30_frames_per_label.groupby("label").std(numeric_only=True)

    # NaNを含む行を削除
    std_values_per_label = std_values_per_label.dropna(axis=0)
    
    # Yを追加
    std_values_per_label["Y"] = labels[i]
    i = i + 1
    
    # 結果をリストに保存
    result_list.append((file_name, std_values_per_label))

# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30.csv:
          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer1     0.001006  0.060736  0.087688  0.092299  0.082711  0.035892   
answer2     0.001069  0.061230  0.085326  0.112583  0.056468  0.046943   
answer3     0.001248  0.108813  0.148984  0.119428  0.065117  0.037821   
answer4     0.001489  0.076634  0.083428  0.071551  0.108240  0.034433   
answer5     0.001968  0.077905  0.102497  0.114414  0.087968  0.041780   
answer6     0.001509  0.054927  0.111568  0.089369  0.089529  0.021638   
answer7     0.001398  0.086977  0.119253  0.127055  0.163408  0.042158   
answer8     0.001651  0.064065  0.075687  0.105997  0.083067  0.034253   
answer9     0.000932  0.113299  0.088482  0.115859  0.074840  0.021622   
answer10    0.002075  0.070181  0.094008  0.116095  0.081840  0.020121   

              AU07      AU09      AU10      AU11  ...      AU15      AU17  \
lab

          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer1     0.000086  0.047001  0.092149  0.065566  0.170823  0.043933   
answer2     0.000014  0.103077  0.035826  0.074794  0.079524  0.046360   
answer3     0.000014  0.069751  0.022281  0.084735  0.065182  0.034162   
answer4     0.000020  0.085884  0.060510  0.048810  0.068051  0.063321   
answer5     0.000022  0.064000  0.065371  0.067291  0.066835  0.032864   
answer6     0.000018  0.082353  0.068442  0.098904  0.021545  0.035120   
answer7     0.000050  0.049093  0.092454  0.119439  0.029052  0.053067   
answer8     0.000050  0.058522  0.049415  0.084136  0.051637  0.034913   
answer9     0.000032  0.056620  0.044717  0.085003  0.020049  0.013589   
answer10    0.000029  0.076077  0.034677  0.124660  0.026427  0.059562   

              AU07      AU09      AU10      AU11  ...      AU15      AU17  \
label                             

          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer1     0.004704  0.077047  0.123011  0.212966  0.076134  0.099532   
answer2     0.005487  0.087247  0.061430  0.193960  0.076546  0.092635   
answer3     0.008234  0.148928  0.194516  0.179961  0.072076  0.093497   
answer5     0.016900  0.117160  0.141554  0.174913  0.119573  0.044549   
answer7     0.015415  0.139607  0.194420  0.202861  0.082893  0.084516   
answer9     0.006239  0.112995  0.161047  0.224397  0.054239  0.114764   
answer10    0.007301  0.047690  0.109408  0.142815  0.059447  0.074175   

              AU07      AU09      AU10      AU11  ...      AU15      AU17  \
label                                             ...                       
answer1   0.517549  0.101415  0.224915  0.517549  ...  0.139480  0.066221   
answer2   0.316228  0.090319  0.294438  0.527046  ...  0.107907  0.072365   
answer3   0.421637  0.115

In [13]:
# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []
correct_labels_list = []  # 正解ラベルを保存するリスト

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # XGBoostモデルの初期化と訓練
    model = xgb.XGBClassifier(eval_metric='logloss')
    model.fit(X_train_combined, y_train_combined)
    
    # 予測
    y_pred = model.predict(X_test)
    
    # 精度計算
    correct_predictions = (y_pred == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100  # 予測率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    correct_labels_list.append(y_test)  # 正解ラベルも保存
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.iloc[0]}")  # 最初の正解ラベルのみ表示
    print(f"予測ラベル: {y_pred}")  # 予測ラベル全体を表示

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = 0
for (file_name, acc), y_test in zip(accuracy_list, correct_labels_list):
    if 1 in y_test.values and acc >= 50:  # 正解ラベルが1を含んでいる場合
        count_above_50_for_label_1 += 1

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
ファイル名: filtered_ID30.csv
予測率: 80.00%
正解ラベル: 0
予測ラベル: [0 0 0 0 0 0 0 1 1 0]

===== テストデータ: ファイル 2/31 =====
ファイル名: filtered_ID31.csv
予測率: 70.00%
正解ラベル: 0
予測ラベル: [0 1 0 1 0 0 0 0 0 1]

===== テストデータ: ファイル 3/31 =====
ファイル名: filtered_ID32.csv
予測率: 20.00%
正解ラベル: 1
予測ラベル: [0 0 0 0 1 0 1 0 0 0]

===== テストデータ: ファイル 4/31 =====
ファイル名: filtered_ID33.csv
予測率: 80.00%
正解ラベル: 0
予測ラベル: [1 0 0 0 1 0 0 0 0 0]

===== テストデータ: ファイル 5/31 =====
ファイル名: filtered_ID34.csv
予測率: 12.50%
正解ラベル: 1
予測ラベル: [0 0 0 1 0 0 0 0]

===== テストデータ: ファイル 6/31 =====
ファイル名: filtered_ID35.csv
予測率: 100.00%
正解ラベル: 0
予測ラベル: [0 0 0 0 0 0 0 0 0]

===== テストデータ: ファイル 7/31 =====
ファイル名: filtered_ID36.csv
予測率: 80.00%
正解ラベル: 0
予測ラベル: [1 0 0 0 1 0 0 0 0 0]

===== テストデータ: ファイル 8/31 =====
ファイル名: filtered_ID37.csv
予測率: 70.00%
正解ラベル: 0
予測ラベル: [0 0 1 0 1 1 0 0 0 0]

===== テストデータ: ファイル 9/31 =====
ファイル名: filtered_ID38.csv
予測率: 90.00%
正解ラベル: 0
予測ラベル: [0 0 0 0 0 1 0 0 0 0]

===== テストデータ: ファイル 10/31 =====
ファイル名: filtered_ID3

In [4]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []
correct_labels_list = []  # 正解ラベルを保存するリスト

# グリッドサーチ用のパラメータ設定
param_grid = {
    'max_depth': [3, 5, 7],           # 木の最大深さ
    'learning_rate': [0.1, 0.01],     # 学習率
    'n_estimators': [50, 100, 200],   # 木の数
    'subsample': [0.8, 1.0],          # サブサンプリング比率
    'colsample_bytree': [0.8, 1.0],   # 特徴量サンプリング比率
}

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    test_df_cleaned = test_df_cleaned.dropna(axis=0)  # NaNを含む行を削除
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            train_df_cleaned = train_df_cleaned.dropna(axis=0)  # NaNを含む行を削除
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # XGBoostモデルをグリッドサーチでチューニング
    xgb_model = xgb.XGBClassifier(eval_metric='logloss')
    grid_search = GridSearchCV(
        estimator=xgb_model,
        param_grid=param_grid,
        scoring='accuracy',   # 評価指標は正答率
        cv=3,                 # 3分割交差検証
        n_jobs=-1,            # 並列処理
        verbose=1             # 進捗状況を表示
    )
    
    # グリッドサーチ実行
    grid_search.fit(X_train_combined, y_train_combined)
    best_model = grid_search.best_estimator_  # 最良モデルを取得
    
    # 予測
    y_pred = best_model.predict(X_test)
    
    # 精度計算
    correct_predictions = (y_pred == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100  # 予測率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    correct_labels_list.append(y_test)  # 正解ラベルも保存
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"最適パラメータ: {grid_search.best_params_}")
    print(f"正解ラベル: {y_test.iloc[0]}")
    print(f"予測ラベル: {y_pred}")

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = 0
for (file_name, acc), y_test in zip(accuracy_list, correct_labels_list):
    if 1 in y_test.values and acc >= 50:  # 正解ラベルが1を含んでいる場合
        count_above_50_for_label_1 += 1

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
Fitting 3 folds for each of 72 candidates, totalling 216 fits
ファイル名: filtered_ID30.csv
予測率: 100.00%
最適パラメータ: {'colsample_bytree': 1.0, 'learning_rate': 0.01, 'max_depth': 7, 'n_estimators': 50, 'subsample': 0.8}
正解ラベル: 0
予測ラベル: [0 0 0 0 0 0 0 0 0 0]

===== テストデータ: ファイル 2/31 =====
Fitting 3 folds for each of 72 candidates, totalling 216 fits
ファイル名: filtered_ID31.csv
予測率: 100.00%
最適パラメータ: {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 7, 'n_estimators': 50, 'subsample': 1.0}
正解ラベル: 0
予測ラベル: [0 0 0 0 0 0 0 0 0 0]

===== テストデータ: ファイル 3/31 =====
Fitting 3 folds for each of 72 candidates, totalling 216 fits
ファイル名: filtered_ID32.csv
予測率: 20.00%
最適パラメータ: {'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100, 'subsample': 1.0}
正解ラベル: 1
予測ラベル: [0 0 0 0 1 0 1 0 0 0]

===== テストデータ: ファイル 4/31 =====
Fitting 3 folds for each of 72 candidates, totalling 216 fits
ファイル名: filtered_ID33.csv
予測率: 100.00%
最適パラメータ: {'colsample_bytree

ファイル名: filtered_ID73.csv
予測率: 100.00%
最適パラメータ: {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 50, 'subsample': 0.8}
正解ラベル: 0
予測ラベル: [0 0 0 0 0 0 0 0 0]

===== 全ファイルごとの予測率 =====
ファイル名: filtered_ID30.csv, 予測率: 100.00%
ファイル名: filtered_ID31.csv, 予測率: 100.00%
ファイル名: filtered_ID32.csv, 予測率: 20.00%
ファイル名: filtered_ID33.csv, 予測率: 100.00%
ファイル名: filtered_ID34.csv, 予測率: 0.00%
ファイル名: filtered_ID35.csv, 予測率: 100.00%
ファイル名: filtered_ID36.csv, 予測率: 80.00%
ファイル名: filtered_ID37.csv, 予測率: 100.00%
ファイル名: filtered_ID38.csv, 予測率: 100.00%
ファイル名: filtered_ID39.csv, 予測率: 100.00%
ファイル名: filtered_ID40.csv, 予測率: 100.00%
ファイル名: filtered_ID41.csv, 予測率: 90.00%
ファイル名: filtered_ID42.csv, 予測率: 100.00%
ファイル名: filtered_ID43.csv, 予測率: 10.00%
ファイル名: filtered_ID44.csv, 予測率: 0.00%
ファイル名: filtered_ID45.csv, 予測率: 100.00%
ファイル名: filtered_ID46.csv, 予測率: 10.00%
ファイル名: filtered_ID47.csv, 予測率: 77.78%
ファイル名: filtered_ID48.csv, 予測率: 40.00%
ファイル名: filtered_ID49.csv, 予測率: 83.33%
ファイル名: filtered_ID50.

## xgboost（先頭30フレームの差分）

In [16]:
# 処理するフォルダのパス
folder_path = './output_filtered_questions'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `answer1`から`answer10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^answer(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"answer{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の30フレームを取得
    first_30_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(30))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )
    
    # ラベルごとにフレーム間の差分を計算
    diff_values_per_label = (
        first_30_frames_per_label.groupby("label")
        .apply(lambda group: group.iloc[:, 1:].diff().dropna())
    )

    # 差分に対応するラベル列をリセット（ラベルを戻す）
    diff_values_per_label = diff_values_per_label.reset_index(level=0).dropna()
    
    # 結果をリストに保存
    result_list.append((file_name, diff_values_per_label))

    # Yを追加
    diff_values_per_label["Y"] = labels[i]
    i = i + 1
    
# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30.csv:
        label  Confidence      AU01      AU02      AU04      AU05      AU06  \
1     answer1    0.002523  0.020600 -0.051877  0.051471 -0.174598 -0.029496   
2     answer1   -0.000777 -0.073554  0.019585 -0.076175  0.157162 -0.012325   
3     answer1   -0.000558  0.192474  0.187477  0.188690 -0.160535 -0.029967   
4     answer1    0.000219 -0.134605  0.036094 -0.201379  0.035716 -0.048250   
5     answer1    0.000912 -0.076099 -0.020339 -0.079744 -0.010129  0.118473   
..        ...         ...       ...       ...       ...       ...       ...   
295  answer10    0.004307 -0.049486 -0.193434  0.265313 -0.097888  0.049583   
296  answer10   -0.000085 -0.013194  0.201371  0.034599 -0.065208 -0.032655   
297  answer10   -0.003561 -0.043562  0.026955 -0.043254  0.008891  0.007323   
298  answer10    0.001567  0.025185 -0.023358 -0.149872 -0.002337  0.024066   
299  answer10    0.001315  0.021714 -0.139229  0.103601 -0.024386 -0.027206   

     AU07      AU09 

        label  Confidence      AU01      AU02      AU04      AU05      AU06  \
1     answer2   -0.001068 -0.118162  0.119186  0.238182  0.128947  0.043439   
2     answer2   -0.002121 -0.009223 -0.323588 -0.117032 -0.115605 -0.109911   
3     answer2   -0.000185  0.107667  0.050007 -0.001902  0.073402  0.238468   
4     answer2   -0.001228 -0.121371 -0.004028 -0.027130 -0.032169 -0.220457   
5     answer2   -0.000241  0.152454  0.169941  0.105927 -0.006818  0.030061   
..        ...         ...       ...       ...       ...       ...       ...   
226  answer10    0.000815 -0.022658  0.009186  0.330684 -0.026439 -0.118581   
227  answer10   -0.000229  0.056954 -0.039308 -0.280139  0.083397  0.014894   
228  answer10    0.000636 -0.085737  0.052603  0.109441  0.009841 -0.004918   
229  answer10    0.000833 -0.228789 -0.194767 -0.220029 -0.156187  0.049086   
230  answer10   -0.001944 -0.007204 -0.075618  0.185340 -0.002289 -0.010773   

     AU07      AU09      AU10  ...      AU15      A

In [17]:
# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []
correct_labels_list = []  # 正解ラベルを保存するリスト

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # XGBoostモデルの初期化と訓練
    model = xgb.XGBClassifier(eval_metric='logloss')
    model.fit(X_train_combined, y_train_combined)
    
    # 予測
    y_pred = model.predict(X_test)
    
    # 精度計算
    correct_predictions = (y_pred == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100  # 予測率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    correct_labels_list.append(y_test)  # 正解ラベルも保存
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.iloc[0]}")  # 最初の正解ラベルのみ表示
    print(f"予測ラベル: {y_pred}")  # 予測ラベル全体を表示

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = 0
for (file_name, acc), y_test in zip(accuracy_list, correct_labels_list):
    if 1 in y_test.values and acc >= 50:  # 正解ラベルが1を含んでいる場合
        count_above_50_for_label_1 += 1

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
ファイル名: filtered_ID30.csv
予測率: 78.97%
正解ラベル: 0
予測ラベル: [0 0 0 1 0 1 1 1 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0 0 1 0 0 0 0 0 0 1 0
 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 1 0 0
 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 1 0 1 0 1 1 1 1 1 0 0 0 0 1 0 0 1 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
 0 0 1 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 1 0
 0 0 0 0 0 0 0 1 0 0 1 0 0 0 1 1 1 0 0 0 0 0 1 0 1 1 0 0 0 0 1 0 1 0 0 1 0
 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0]

===== テストデータ: ファイル 2/31 =====
ファイル名: filtered_ID31.csv
予測率: 72.39%
正解ラベル: 0
予測ラベル: [1 1 0 1 0 0 0 1 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0
 1 0 0 0 0 0 0 1 0 0 0 1 0 0 1 1 0 0 0 1 0 1 0 0 1 0 1 0 1 0 1 1 0 1 0 0 0
 1 0 0 0 1 1 1 0 1 1 0 1 1 1 0 1 0 0 0 0 1 0 0 0 1 0 1 0 1 1 0 1 0 1 0 0 0
 0 0 1 1 1 0 0 0 0

ファイル名: filtered_ID44.csv
予測率: 14.20%
正解ラベル: 1
予測ラベル: [0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0
 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1 1 0 0 0 0 0 1 1 0 0 0 0 0 1 1 0 0
 0 0 0 0 0 1 0 0 0 1 0 0 0 0]

===== テストデータ: ファイル 16/31 =====
ファイル名: filtered_ID45.csv
予測率: 86.49%
正解ラベル: 0
予測ラベル: [0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
 0 0 0 0 0 1 1 0 0 0 1 0 1 0 0 1 0 1 1 0 0 0 0 0 1 0 0 1 1 0 0 1 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0
 0 1 0 1 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]

===== テストデータ: ファイル 17/31 =====
ファイル名: filtered_ID46.csv
予測率: 32.27%
正解ラベル: 1
予測ラ

ファイル名: filtered_ID73.csv
予測率: 76.92%
正解ラベル: 0
予測ラベル: [0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 1 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 1 1 1 1 0 1 0 0 0 0 0 0 0 0 0 1 1
 0 0 0 1 0 1 0 1 1 1 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 1 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 1 1 1 0 0 0 1 1 0 1 0 0 0]

===== 全ファイルごとの予測率 =====
ファイル名: filtered_ID30.csv, 予測率: 78.97%
ファイル名: filtered_ID31.csv, 予測率: 72.39%
ファイル名: filtered_ID32.csv, 予測率: 15.52%
ファイル名: filtered_ID33.csv, 予測率: 76.12%
ファイル名: filtered_ID34.csv, 予測率: 25.47%
ファイル名: filtered_ID35.csv, 予測率: 75.88%
ファイル名: filtered_ID36.csv, 予測率: 85.17%
ファイル名: filtered_ID37.csv, 予測率: 90.42%
ファイル名: filtered_ID38.csv, 予測率: 80.51%
ファイル名: filtered_ID39.csv, 予測率: 81.45%
ファイル名: filtered_ID40.csv, 予測率: 87.60%
ファイル名: filtered_ID41.csv, 予測率: 82.66%
ファイル名: filtered_I

## xgboost（先頭30フレームの差分の標準偏差）

In [9]:
# 処理するフォルダのパス
folder_path = './output_filtered_questions'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `answer1`から`answer10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^answer(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"answer{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の30フレームを取得
    first_30_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(30))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )
    
    # ラベルごとにフレーム間の差分を計算
    diff_values_per_label = (
        first_30_frames_per_label.groupby("label")
        .apply(lambda group: group.iloc[:, 1:].diff().dropna())
    )

    # 差分に対応するラベル列をリセット（ラベルを戻す）
    diff_values_per_label = diff_values_per_label.reset_index(level=0).dropna()
    
    # 差分の標準偏差を計算
    std_values_per_label = (
        diff_values_per_label.groupby("label").std(numeric_only=True)
    )
    
    # 結果をリストに保存
    result_list.append((file_name, std_values_per_label))

    # Yを追加
    std_values_per_label["Y"] = labels[i]
    i = i + 1
    
# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30.csv:
          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer1     0.001462  0.094932  0.099262  0.166714  0.088797  0.052320   
answer2     0.001210  0.112730  0.110444  0.150327  0.068664  0.052901   
answer3     0.001548  0.116955  0.136898  0.151067  0.093528  0.041603   
answer4     0.001596  0.097164  0.089056  0.138602  0.157043  0.047489   
answer5     0.002208  0.070803  0.122991  0.145819  0.142746  0.044949   
answer6     0.001575  0.063542  0.092813  0.169444  0.153299  0.035713   
answer7     0.001426  0.089564  0.160565  0.190157  0.215378  0.040423   
answer8     0.001579  0.099095  0.110874  0.151145  0.077755  0.037964   
answer9     0.001348  0.106613  0.140040  0.136975  0.058231  0.043743   
answer10    0.002133  0.072338  0.109438  0.161577  0.077560  0.034574   

              AU07      AU09      AU10      AU11  ...      AU15      AU17  \
lab

          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer1     0.003334  0.174561  0.155981  0.238468  0.078974  0.067495   
answer2     0.000452  0.187483  0.194865  0.259664  0.060766  0.010332   
answer3     0.000644  0.271308  0.258594  0.334830  0.051546  0.172812   
answer4     0.001184  0.205033  0.257118  0.276463  0.057358  0.020656   
answer5     0.000516  0.308396  0.451421  0.370528  0.083615  0.102296   
answer6     0.000839  0.210294  0.259140  0.372651  0.069385  0.101167   
answer7     0.000596  0.285835  0.287972  0.470774  0.083620  0.033422   
answer8     0.001233  0.363062  0.277969  0.394487  0.106473  0.027973   
answer9     0.000934  0.236547  0.239629  0.333982  0.061768  0.013561   
answer10    0.000809  0.201287  0.238693  0.402473  0.072859  0.100476   

              AU07      AU09      AU10      AU11  ...      AU15      AU17  \
label                             

In [19]:
# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []
correct_labels_list = []  # 正解ラベルを保存するリスト

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # XGBoostモデルの初期化と訓練
    model = xgb.XGBClassifier(eval_metric='logloss')
    model.fit(X_train_combined, y_train_combined)
    
    # 予測
    y_pred = model.predict(X_test)
    
    # 精度計算
    correct_predictions = (y_pred == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100  # 予測率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    correct_labels_list.append(y_test)  # 正解ラベルも保存
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.iloc[0]}")  # 最初の正解ラベルのみ表示
    print(f"予測ラベル: {y_pred}")  # 予測ラベル全体を表示

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = 0
for (file_name, acc), y_test in zip(accuracy_list, correct_labels_list):
    if 1 in y_test.values and acc >= 50:  # 正解ラベルが1を含んでいる場合
        count_above_50_for_label_1 += 1

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
ファイル名: filtered_ID30.csv
予測率: 100.00%
正解ラベル: 0
予測ラベル: [0 0 0 0 0 0 0 0 0 0]

===== テストデータ: ファイル 2/31 =====
ファイル名: filtered_ID31.csv
予測率: 80.00%
正解ラベル: 0
予測ラベル: [1 0 1 0 0 0 0 0 0 0]

===== テストデータ: ファイル 3/31 =====
ファイル名: filtered_ID32.csv
予測率: 0.00%
正解ラベル: 1
予測ラベル: [0 0 0 0 0 0 0 0 0 0]

===== テストデータ: ファイル 4/31 =====
ファイル名: filtered_ID33.csv
予測率: 80.00%
正解ラベル: 0
予測ラベル: [1 0 0 0 1 0 0 0 0 0]

===== テストデータ: ファイル 5/31 =====
ファイル名: filtered_ID34.csv
予測率: 37.50%
正解ラベル: 1
予測ラベル: [0 1 0 0 1 0 1 0]

===== テストデータ: ファイル 6/31 =====
ファイル名: filtered_ID35.csv
予測率: 77.78%
正解ラベル: 0
予測ラベル: [1 0 0 0 1 0 0 0 0]

===== テストデータ: ファイル 7/31 =====
ファイル名: filtered_ID36.csv
予測率: 90.00%
正解ラベル: 0
予測ラベル: [0 0 0 0 1 0 0 0 0 0]

===== テストデータ: ファイル 8/31 =====
ファイル名: filtered_ID37.csv
予測率: 100.00%
正解ラベル: 0
予測ラベル: [0 0 0 0 0 0 0 0 0 0]

===== テストデータ: ファイル 9/31 =====
ファイル名: filtered_ID38.csv
予測率: 80.00%
正解ラベル: 0
予測ラベル: [0 0 0 0 1 0 0 0 0 1]

===== テストデータ: ファイル 10/31 =====
ファイル名: filtered_ID3

In [10]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []
correct_labels_list = []  # 正解ラベルを保存するリスト

# グリッドサーチ用のパラメータ設定
param_grid = {
    'max_depth': [3, 5, 7],           # 木の最大深さ
    'learning_rate': [0.1, 0.01],     # 学習率
    'n_estimators': [50, 100, 200],   # 木の数
    'subsample': [0.8, 1.0],          # サブサンプリング比率
    'colsample_bytree': [0.8, 1.0],   # 特徴量サンプリング比率
}

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    test_df_cleaned = test_df_cleaned.dropna(axis=0)  # NaNを含む行を削除
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            train_df_cleaned = train_df_cleaned.dropna(axis=0)  # NaNを含む行を削除
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # XGBoostモデルをグリッドサーチでチューニング
    xgb_model = xgb.XGBClassifier(eval_metric='logloss')
    grid_search = GridSearchCV(
        estimator=xgb_model,
        param_grid=param_grid,
        scoring='accuracy',   # 評価指標は正答率
        cv=3,                 # 3分割交差検証
        n_jobs=-1,            # 並列処理
        verbose=1             # 進捗状況を表示
    )
    
    # グリッドサーチ実行
    grid_search.fit(X_train_combined, y_train_combined)
    best_model = grid_search.best_estimator_  # 最良モデルを取得
    
    # 予測
    y_pred = best_model.predict(X_test)
    
    # 精度計算
    correct_predictions = (y_pred == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100  # 予測率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    correct_labels_list.append(y_test)  # 正解ラベルも保存
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"最適パラメータ: {grid_search.best_params_}")
    print(f"正解ラベル: {y_test.iloc[0]}")
    print(f"予測ラベル: {y_pred}")

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = 0
for (file_name, acc), y_test in zip(accuracy_list, correct_labels_list):
    if 1 in y_test.values and acc >= 50:  # 正解ラベルが1を含んでいる場合
        count_above_50_for_label_1 += 1

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
Fitting 3 folds for each of 72 candidates, totalling 216 fits
ファイル名: filtered_ID30.csv
予測率: 100.00%
最適パラメータ: {'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 50, 'subsample': 0.8}
正解ラベル: 0
予測ラベル: [0 0 0 0 0 0 0 0 0 0]

===== テストデータ: ファイル 2/31 =====
Fitting 3 folds for each of 72 candidates, totalling 216 fits
ファイル名: filtered_ID31.csv
予測率: 60.00%
最適パラメータ: {'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 50, 'subsample': 0.8}
正解ラベル: 0
予測ラベル: [1 0 1 1 0 0 0 0 1 0]

===== テストデータ: ファイル 3/31 =====
Fitting 3 folds for each of 72 candidates, totalling 216 fits
ファイル名: filtered_ID32.csv
予測率: 0.00%
最適パラメータ: {'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 50, 'subsample': 0.8}
正解ラベル: 1
予測ラベル: [0 0 0 0 0 0 0 0 0 0]

===== テストデータ: ファイル 4/31 =====
Fitting 3 folds for each of 72 candidates, totalling 216 fits
ファイル名: filtered_ID33.csv
予測率: 100.00%
最適パラメータ: {'colsample_bytree': 0.

ファイル名: filtered_ID73.csv
予測率: 88.89%
最適パラメータ: {'colsample_bytree': 1.0, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100, 'subsample': 1.0}
正解ラベル: 0
予測ラベル: [1 0 0 0 0 0 0 0 0]

===== 全ファイルごとの予測率 =====
ファイル名: filtered_ID30.csv, 予測率: 100.00%
ファイル名: filtered_ID31.csv, 予測率: 60.00%
ファイル名: filtered_ID32.csv, 予測率: 0.00%
ファイル名: filtered_ID33.csv, 予測率: 100.00%
ファイル名: filtered_ID34.csv, 予測率: 0.00%
ファイル名: filtered_ID35.csv, 予測率: 77.78%
ファイル名: filtered_ID36.csv, 予測率: 100.00%
ファイル名: filtered_ID37.csv, 予測率: 100.00%
ファイル名: filtered_ID38.csv, 予測率: 100.00%
ファイル名: filtered_ID39.csv, 予測率: 100.00%
ファイル名: filtered_ID40.csv, 予測率: 90.00%
ファイル名: filtered_ID41.csv, 予測率: 100.00%
ファイル名: filtered_ID42.csv, 予測率: 100.00%
ファイル名: filtered_ID43.csv, 予測率: 0.00%
ファイル名: filtered_ID44.csv, 予測率: 0.00%
ファイル名: filtered_ID45.csv, 予測率: 100.00%
ファイル名: filtered_ID46.csv, 予測率: 0.00%
ファイル名: filtered_ID47.csv, 予測率: 22.22%
ファイル名: filtered_ID48.csv, 予測率: 40.00%
ファイル名: filtered_ID49.csv, 予測率: 0.00%
ファイル名: filtered_ID50.csv, 

## xgboost（先頭30フレームの最大値）

In [11]:
# 処理するフォルダのパス
folder_path = './output_filtered_questions'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `answer1`から`answer10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^answer(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"answer{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の30フレームを取得
    first_30_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(30))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )

    # ラベルごとに30フレームの最大値を計算
    max_values_per_label = (
        first_30_frames_per_label.groupby("label").max(numeric_only=True)
    )
    
    # 結果をリストに保存
    result_list.append((file_name, max_values_per_label))

    # Yを追加
    max_values_per_label["Y"] = labels[i]
    i = i + 1
    
# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30.csv:
          Confidence      AU01      AU02      AU04      AU05      AU06  AU07  \
label                                                                          
answer1     0.998552  0.772197  0.470349  0.833798  0.497938  0.281521   1.0   
answer2     0.998392  0.823720  0.504450  0.698435  0.451498  0.273104   1.0   
answer3     0.998105  0.810446  0.634203  0.735056  0.553234  0.224017   1.0   
answer4     0.996902  0.805873  0.572209  0.587676  0.748750  0.229382   1.0   
answer5     0.997551  0.775502  0.627177  0.667268  0.585380  0.243576   1.0   
answer6     0.997832  0.788442  0.728710  0.603687  0.741321  0.221382   1.0   
answer7     0.997573  0.748430  0.691159  0.728610  0.753132  0.290905   1.0   
answer8     0.997223  0.828781  0.607205  0.764085  0.546507  0.219612   1.0   
answer9     0.998192  0.806971  0.617136  0.706027  0.529929  0.314064   1.0   
answer10    0.996892  0.802406  0.600398  0.747122  0.589219  0.245450   1.0   

        

          Confidence      AU01      AU02      AU04      AU05      AU06  AU07  \
label                                                                          
answer1          NaN       NaN       NaN       NaN       NaN       NaN   NaN   
answer2     0.999171  0.577754  0.436342  0.728956  0.406655  0.137717   0.0   
answer3     0.999527  0.723047  0.520864  0.748390  0.406034  0.122219   0.0   
answer4     0.999396  0.801725  0.533584  0.828706  0.367839  0.463472   1.0   
answer5          NaN       NaN       NaN       NaN       NaN       NaN   NaN   
answer6     0.979085  0.788371  0.777610  0.717418  0.449547  0.562481   1.0   
answer7     0.999883  0.884061  0.715420  0.551763  0.753195  0.163728   1.0   
answer8     0.997704  0.724520  0.790253  0.821001  0.431553  0.487582   1.0   
answer9     0.999867  0.566770  0.310816  0.876023  0.432052  0.161897   1.0   
answer10    0.999815  0.720461  0.469627  0.762009  0.422979  0.212527   1.0   

              AU09      AU10  AU11  ...

          Confidence      AU01      AU02      AU04      AU05      AU06  AU07  \
label                                                                          
answer1     0.997757  0.708355  0.490706  0.780773  0.457992  0.189607   1.0   
answer2     0.999097  0.838759  0.633998  0.598772  0.514382  0.163188   1.0   
answer3     0.999137  0.811415  0.671352  0.563651  0.400471  0.285653   1.0   
answer4     0.999543  0.880877  0.674349  0.502733  0.441709  0.279269   1.0   
answer5          NaN       NaN       NaN       NaN       NaN       NaN   NaN   
answer6     0.998091  0.892349  0.655172  0.656235  0.454090  0.360759   1.0   
answer7     0.999233  0.876878  0.676405  0.590513  0.484600  0.246969   1.0   
answer8     0.998802  0.883611  0.646489  0.497106  0.470754  0.475613   1.0   
answer9     0.999001  0.865051  0.576202  0.535037  0.402584  0.236214   1.0   
answer10    0.999221  0.887974  0.631503  0.674590  0.466397  0.596702   1.0   

              AU09      AU10  AU11  ...

In [21]:
# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []
correct_labels_list = []  # 正解ラベルを保存するリスト

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # XGBoostモデルの初期化と訓練
    model = xgb.XGBClassifier(eval_metric='logloss')
    model.fit(X_train_combined, y_train_combined)
    
    # 予測
    y_pred = model.predict(X_test)
    
    # 精度計算
    correct_predictions = (y_pred == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100  # 予測率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    correct_labels_list.append(y_test)  # 正解ラベルも保存
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.iloc[0]}")  # 最初の正解ラベルのみ表示
    print(f"予測ラベル: {y_pred}")  # 予測ラベル全体を表示

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = 0
for (file_name, acc), y_test in zip(accuracy_list, correct_labels_list):
    if 1 in y_test.values and acc >= 50:  # 正解ラベルが1を含んでいる場合
        count_above_50_for_label_1 += 1

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
ファイル名: filtered_ID30.csv
予測率: 100.00%
正解ラベル: 0
予測ラベル: [0 0 0 0 0 0 0 0 0 0]

===== テストデータ: ファイル 2/31 =====
ファイル名: filtered_ID31.csv
予測率: 50.00%
正解ラベル: 0
予測ラベル: [1 1 1 1 0 1 0 0 0 0]

===== テストデータ: ファイル 3/31 =====
ファイル名: filtered_ID32.csv
予測率: 0.00%
正解ラベル: 1
予測ラベル: [0 0 0 0 0 0 0 0 0 0]

===== テストデータ: ファイル 4/31 =====
ファイル名: filtered_ID33.csv
予測率: 80.00%
正解ラベル: 0
予測ラベル: [1 0 0 0 1 0 0 0 0 0]

===== テストデータ: ファイル 5/31 =====
ファイル名: filtered_ID34.csv
予測率: 0.00%
正解ラベル: 1
予測ラベル: [0 0 0 0 0 0 0 0]

===== テストデータ: ファイル 6/31 =====
ファイル名: filtered_ID35.csv
予測率: 77.78%
正解ラベル: 0
予測ラベル: [0 0 0 1 0 0 1 0 0]

===== テストデータ: ファイル 7/31 =====
ファイル名: filtered_ID36.csv
予測率: 30.00%
正解ラベル: 0
予測ラベル: [1 1 0 1 1 0 1 0 1 1]

===== テストデータ: ファイル 8/31 =====
ファイル名: filtered_ID37.csv
予測率: 100.00%
正解ラベル: 0
予測ラベル: [0 0 0 0 0 0 0 0 0 0]

===== テストデータ: ファイル 9/31 =====
ファイル名: filtered_ID38.csv
予測率: 100.00%
正解ラベル: 0
予測ラベル: [0 0 0 0 0 0 0 0 0 0]

===== テストデータ: ファイル 10/31 =====
ファイル名: filtered_ID3

In [12]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []
correct_labels_list = []  # 正解ラベルを保存するリスト

# グリッドサーチ用のパラメータ設定
param_grid = {
    'max_depth': [3, 5, 7],           # 木の最大深さ
    'learning_rate': [0.1, 0.01],     # 学習率
    'n_estimators': [50, 100, 200],   # 木の数
    'subsample': [0.8, 1.0],          # サブサンプリング比率
    'colsample_bytree': [0.8, 1.0],   # 特徴量サンプリング比率
}

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    test_df_cleaned = test_df_cleaned.dropna(axis=0)  # NaNを含む行を削除
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            train_df_cleaned = train_df_cleaned.dropna(axis=0)  # NaNを含む行を削除
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # XGBoostモデルをグリッドサーチでチューニング
    xgb_model = xgb.XGBClassifier(eval_metric='logloss')
    grid_search = GridSearchCV(
        estimator=xgb_model,
        param_grid=param_grid,
        scoring='accuracy',   # 評価指標は正答率
        cv=3,                 # 3分割交差検証
        n_jobs=-1,            # 並列処理
        verbose=1             # 進捗状況を表示
    )
    
    # グリッドサーチ実行
    grid_search.fit(X_train_combined, y_train_combined)
    best_model = grid_search.best_estimator_  # 最良モデルを取得
    
    # 予測
    y_pred = best_model.predict(X_test)
    
    # 精度計算
    correct_predictions = (y_pred == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100  # 予測率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    correct_labels_list.append(y_test)  # 正解ラベルも保存
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"最適パラメータ: {grid_search.best_params_}")
    print(f"正解ラベル: {y_test.iloc[0]}")
    print(f"予測ラベル: {y_pred}")

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = 0
for (file_name, acc), y_test in zip(accuracy_list, correct_labels_list):
    if 1 in y_test.values and acc >= 50:  # 正解ラベルが1を含んでいる場合
        count_above_50_for_label_1 += 1

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
Fitting 3 folds for each of 72 candidates, totalling 216 fits
ファイル名: filtered_ID30.csv
予測率: 100.00%
最適パラメータ: {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 50, 'subsample': 1.0}
正解ラベル: 0
予測ラベル: [0 0 0 0 0 0 0 0 0 0]

===== テストデータ: ファイル 2/31 =====
Fitting 3 folds for each of 72 candidates, totalling 216 fits
ファイル名: filtered_ID31.csv
予測率: 80.00%
最適パラメータ: {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 7, 'n_estimators': 50, 'subsample': 0.8}
正解ラベル: 0
予測ラベル: [0 0 0 1 0 1 0 0 0 0]

===== テストデータ: ファイル 3/31 =====
Fitting 3 folds for each of 72 candidates, totalling 216 fits
ファイル名: filtered_ID32.csv
予測率: 0.00%
最適パラメータ: {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 50, 'subsample': 0.8}
正解ラベル: 1
予測ラベル: [0 0 0 0 0 0 0 0 0 0]

===== テストデータ: ファイル 4/31 =====
Fitting 3 folds for each of 72 candidates, totalling 216 fits
ファイル名: filtered_ID33.csv
予測率: 100.00%
最適パラメータ: {'colsample_bytree': 

ファイル名: filtered_ID73.csv
予測率: 100.00%
最適パラメータ: {'colsample_bytree': 1.0, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 50, 'subsample': 1.0}
正解ラベル: 0
予測ラベル: [0 0 0 0 0 0 0 0 0]

===== 全ファイルごとの予測率 =====
ファイル名: filtered_ID30.csv, 予測率: 100.00%
ファイル名: filtered_ID31.csv, 予測率: 80.00%
ファイル名: filtered_ID32.csv, 予測率: 0.00%
ファイル名: filtered_ID33.csv, 予測率: 100.00%
ファイル名: filtered_ID34.csv, 予測率: 0.00%
ファイル名: filtered_ID35.csv, 予測率: 77.78%
ファイル名: filtered_ID36.csv, 予測率: 80.00%
ファイル名: filtered_ID37.csv, 予測率: 100.00%
ファイル名: filtered_ID38.csv, 予測率: 100.00%
ファイル名: filtered_ID39.csv, 予測率: 100.00%
ファイル名: filtered_ID40.csv, 予測率: 90.00%
ファイル名: filtered_ID41.csv, 予測率: 100.00%
ファイル名: filtered_ID42.csv, 予測率: 100.00%
ファイル名: filtered_ID43.csv, 予測率: 0.00%
ファイル名: filtered_ID44.csv, 予測率: 0.00%
ファイル名: filtered_ID45.csv, 予測率: 87.50%
ファイル名: filtered_ID46.csv, 予測率: 0.00%
ファイル名: filtered_ID47.csv, 予測率: 55.56%
ファイル名: filtered_ID48.csv, 予測率: 40.00%
ファイル名: filtered_ID49.csv, 予測率: 66.67%
ファイル名: filtered_ID50.csv, 予

## xgboost（先頭10フレームの最大値）

In [13]:
# 処理するフォルダのパス
folder_path = './output_filtered_questions'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `answer1`から`answer10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^answer(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"answer{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の10フレームを取得
    first_10_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(10))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )

    # ラベルごとに10フレームの最大値を計算
    max_values_per_label = (
        first_10_frames_per_label.groupby("label").max(numeric_only=True)
    )
    
    # 結果をリストに保存
    result_list.append((file_name, max_values_per_label))

    # Yを追加
    max_values_per_label["Y"] = labels[i]
    i = i + 1
    
# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30.csv:
          Confidence      AU01      AU02      AU04      AU05      AU06  AU07  \
label                                                                          
answer1     0.998552  0.740146  0.404279  0.597930  0.460274  0.214406   1.0   
answer2     0.998392  0.685272  0.488080  0.692978  0.420650  0.256867   1.0   
answer3     0.998105  0.737937  0.634203  0.735056  0.459632  0.214803   1.0   
answer4     0.996902  0.772896  0.464763  0.581556  0.555469  0.215759   1.0   
answer5     0.996959  0.757068  0.453117  0.667268  0.574917  0.243576   1.0   
answer6     0.997096  0.743668  0.728710  0.480454  0.553200  0.191910   0.0   
answer7     0.997573  0.748430  0.528374  0.656478  0.752446  0.290905   1.0   
answer8     0.997107  0.749335  0.464386  0.738110  0.546507  0.218950   1.0   
answer9     0.996756  0.806971  0.539072  0.625298  0.522182  0.207981   0.0   
answer10    0.996236  0.802406  0.600398  0.590113  0.551615  0.214269   1.0   

        

          Confidence      AU01      AU02      AU04      AU05      AU06  AU07  \
label                                                                          
answer1          NaN       NaN       NaN       NaN       NaN       NaN   NaN   
answer2     0.998586  0.577326  0.436342  0.686767  0.373073  0.105176   0.0   
answer3     0.999527  0.723047  0.520864  0.748390  0.376866  0.122219   0.0   
answer4     0.999041  0.691710  0.472567  0.735450  0.332093  0.107767   0.0   
answer5          NaN       NaN       NaN       NaN       NaN       NaN   NaN   
answer6     0.979085  0.732281  0.777610  0.517694  0.376316  0.562481   1.0   
answer7     0.999862  0.666109  0.440438  0.551763  0.597022  0.127970   1.0   
answer8     0.997409  0.724520  0.790253  0.821001  0.404478  0.487582   1.0   
answer9     0.999867  0.434373  0.281635  0.876023  0.425659  0.161897   1.0   
answer10    0.999762  0.720461  0.469627  0.762009  0.422979  0.212527   1.0   

              AU09      AU10  AU11  ...

In [23]:
# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []
correct_labels_list = []  # 正解ラベルを保存するリスト

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # XGBoostモデルの初期化と訓練
    model = xgb.XGBClassifier(eval_metric='logloss')
    model.fit(X_train_combined, y_train_combined)
    
    # 予測
    y_pred = model.predict(X_test)
    
    # 精度計算
    correct_predictions = (y_pred == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100  # 予測率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    correct_labels_list.append(y_test)  # 正解ラベルも保存
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.iloc[0]}")  # 最初の正解ラベルのみ表示
    print(f"予測ラベル: {y_pred}")  # 予測ラベル全体を表示

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = 0
for (file_name, acc), y_test in zip(accuracy_list, correct_labels_list):
    if 1 in y_test.values and acc >= 50:  # 正解ラベルが1を含んでいる場合
        count_above_50_for_label_1 += 1

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
ファイル名: filtered_ID30.csv
予測率: 70.00%
正解ラベル: 0
予測ラベル: [0 0 0 0 1 1 1 0 0 0]

===== テストデータ: ファイル 2/31 =====
ファイル名: filtered_ID31.csv
予測率: 60.00%
正解ラベル: 0
予測ラベル: [0 1 0 1 0 1 0 0 0 1]

===== テストデータ: ファイル 3/31 =====
ファイル名: filtered_ID32.csv
予測率: 0.00%
正解ラベル: 1
予測ラベル: [0 0 0 0 0 0 0 0 0 0]

===== テストデータ: ファイル 4/31 =====
ファイル名: filtered_ID33.csv
予測率: 90.00%
正解ラベル: 0
予測ラベル: [1 0 0 0 0 0 0 0 0 0]

===== テストデータ: ファイル 5/31 =====
ファイル名: filtered_ID34.csv
予測率: 25.00%
正解ラベル: 1
予測ラベル: [1 0 0 0 0 1 0 0]

===== テストデータ: ファイル 6/31 =====
ファイル名: filtered_ID35.csv
予測率: 88.89%
正解ラベル: 0
予測ラベル: [0 0 0 1 0 0 0 0 0]

===== テストデータ: ファイル 7/31 =====
ファイル名: filtered_ID36.csv
予測率: 50.00%
正解ラベル: 0
予測ラベル: [1 0 1 0 1 0 0 0 1 1]

===== テストデータ: ファイル 8/31 =====
ファイル名: filtered_ID37.csv
予測率: 100.00%
正解ラベル: 0
予測ラベル: [0 0 0 0 0 0 0 0 0 0]

===== テストデータ: ファイル 9/31 =====
ファイル名: filtered_ID38.csv
予測率: 100.00%
正解ラベル: 0
予測ラベル: [0 0 0 0 0 0 0 0 0 0]

===== テストデータ: ファイル 10/31 =====
ファイル名: filtered_ID3

In [14]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []
correct_labels_list = []  # 正解ラベルを保存するリスト

# グリッドサーチ用のパラメータ設定
param_grid = {
    'max_depth': [3, 5, 7],           # 木の最大深さ
    'learning_rate': [0.1, 0.01],     # 学習率
    'n_estimators': [50, 100, 200],   # 木の数
    'subsample': [0.8, 1.0],          # サブサンプリング比率
    'colsample_bytree': [0.8, 1.0],   # 特徴量サンプリング比率
}

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    test_df_cleaned = test_df_cleaned.dropna(axis=0)  # NaNを含む行を削除
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            train_df_cleaned = train_df_cleaned.dropna(axis=0)  # NaNを含む行を削除
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # XGBoostモデルをグリッドサーチでチューニング
    xgb_model = xgb.XGBClassifier(eval_metric='logloss')
    grid_search = GridSearchCV(
        estimator=xgb_model,
        param_grid=param_grid,
        scoring='accuracy',   # 評価指標は正答率
        cv=3,                 # 3分割交差検証
        n_jobs=-1,            # 並列処理
        verbose=1             # 進捗状況を表示
    )
    
    # グリッドサーチ実行
    grid_search.fit(X_train_combined, y_train_combined)
    best_model = grid_search.best_estimator_  # 最良モデルを取得
    
    # 予測
    y_pred = best_model.predict(X_test)
    
    # 精度計算
    correct_predictions = (y_pred == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100  # 予測率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    correct_labels_list.append(y_test)  # 正解ラベルも保存
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"最適パラメータ: {grid_search.best_params_}")
    print(f"正解ラベル: {y_test.iloc[0]}")
    print(f"予測ラベル: {y_pred}")

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = 0
for (file_name, acc), y_test in zip(accuracy_list, correct_labels_list):
    if 1 in y_test.values and acc >= 50:  # 正解ラベルが1を含んでいる場合
        count_above_50_for_label_1 += 1

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
Fitting 3 folds for each of 72 candidates, totalling 216 fits
ファイル名: filtered_ID30.csv
予測率: 100.00%
最適パラメータ: {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 7, 'n_estimators': 50, 'subsample': 0.8}
正解ラベル: 0
予測ラベル: [0 0 0 0 0 0 0 0 0 0]

===== テストデータ: ファイル 2/31 =====
Fitting 3 folds for each of 72 candidates, totalling 216 fits
ファイル名: filtered_ID31.csv
予測率: 90.00%
最適パラメータ: {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 7, 'n_estimators': 50, 'subsample': 0.8}
正解ラベル: 0
予測ラベル: [0 1 0 0 0 0 0 0 0 0]

===== テストデータ: ファイル 3/31 =====
Fitting 3 folds for each of 72 candidates, totalling 216 fits
ファイル名: filtered_ID32.csv
予測率: 0.00%
最適パラメータ: {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 50, 'subsample': 0.8}
正解ラベル: 1
予測ラベル: [0 0 0 0 0 0 0 0 0 0]

===== テストデータ: ファイル 4/31 =====
Fitting 3 folds for each of 72 candidates, totalling 216 fits
ファイル名: filtered_ID33.csv
予測率: 100.00%
最適パラメータ: {'colsample_bytree': 

ファイル名: filtered_ID73.csv
予測率: 77.78%
最適パラメータ: {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 50, 'subsample': 0.8}
正解ラベル: 0
予測ラベル: [0 0 1 0 0 0 0 0 1]

===== 全ファイルごとの予測率 =====
ファイル名: filtered_ID30.csv, 予測率: 100.00%
ファイル名: filtered_ID31.csv, 予測率: 90.00%
ファイル名: filtered_ID32.csv, 予測率: 0.00%
ファイル名: filtered_ID33.csv, 予測率: 100.00%
ファイル名: filtered_ID34.csv, 予測率: 0.00%
ファイル名: filtered_ID35.csv, 予測率: 100.00%
ファイル名: filtered_ID36.csv, 予測率: 80.00%
ファイル名: filtered_ID37.csv, 予測率: 100.00%
ファイル名: filtered_ID38.csv, 予測率: 100.00%
ファイル名: filtered_ID39.csv, 予測率: 100.00%
ファイル名: filtered_ID40.csv, 予測率: 80.00%
ファイル名: filtered_ID41.csv, 予測率: 100.00%
ファイル名: filtered_ID42.csv, 予測率: 100.00%
ファイル名: filtered_ID43.csv, 予測率: 40.00%
ファイル名: filtered_ID44.csv, 予測率: 0.00%
ファイル名: filtered_ID45.csv, 予測率: 87.50%
ファイル名: filtered_ID46.csv, 予測率: 0.00%
ファイル名: filtered_ID47.csv, 予測率: 66.67%
ファイル名: filtered_ID48.csv, 予測率: 70.00%
ファイル名: filtered_ID49.csv, 予測率: 66.67%
ファイル名: filtered_ID50.csv, 予

## xgboost（先頭30フレームの最大値と最小値の差分）

In [5]:
# 処理するフォルダのパス
folder_path = './output_filtered_questions'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `answer1`から`answer10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^answer(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"answer{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の30フレームを取得
    first_30_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(30))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )

    # ラベルごとに最大値を計算
    max_values_per_label = (
        first_30_frames_per_label.groupby("label").max(numeric_only=True)
    )

    # ラベルごとに最小値を計算
    min_values_per_label = (
        first_30_frames_per_label.groupby("label").min(numeric_only=True)
    )

    # 最大値と最小値の差分を計算
    range_values_per_label = max_values_per_label - min_values_per_label

    
    # 結果をリストに保存
    result_list.append((file_name, range_values_per_label))

    # Yを追加
    range_values_per_label["Y"] = labels[i]
    i = i + 1
    
# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30.csv:
          Confidence      AU01      AU02      AU04      AU05      AU06  AU07  \
label                                                                          
answer1     0.006316  0.254691  0.337912  0.633875  0.265135  0.187152   1.0   
answer2     0.003334  0.360066  0.284054  0.404317  0.227819  0.160198   1.0   
answer3     0.005948  0.412699  0.485027  0.423869  0.311095  0.129310   1.0   
answer4     0.004306  0.272166  0.359251  0.414689  0.485407  0.142035   1.0   
answer5     0.006226  0.248887  0.445189  0.463946  0.334733  0.152792   1.0   
answer6     0.005584  0.225375  0.392590  0.401407  0.457584  0.112775   1.0   
answer7     0.005535  0.268238  0.529722  0.553741  0.478301  0.185603   1.0   
answer8     0.006327  0.262135  0.358655  0.426853  0.292341  0.105205   1.0   
answer9     0.004578  0.412329  0.461215  0.456385  0.231980  0.178472   1.0   
answer10    0.006379  0.234590  0.290516  0.518986  0.305036  0.095292   1.0   

        

          Confidence      AU01      AU02      AU04      AU05      AU06  AU07  \
label                                                                          
answer1     0.006594  0.155740  0.323634  0.432452  0.381643  0.196697   1.0   
answer2     0.000282  0.378685  0.214642  0.257595  0.248574  0.181980   1.0   
answer3     0.000079  0.270455  0.310878  0.313703  0.230743  0.260576   1.0   
answer4     0.000083  0.356510  0.313291  0.401054  0.307321  0.265281   1.0   
answer5     0.000113  0.331242  0.337684  0.579791  0.378020  0.206271   1.0   
answer6     0.000074  0.314113  0.322690  0.419065  0.198416  0.237051   1.0   
answer7     0.000191  0.358053  0.321393  0.416862  0.207203  0.212927   1.0   
answer8     0.000223  0.228310  0.235759  0.437042  0.272814  0.352357   1.0   
answer9     0.000147  0.241841  0.218457  0.385809  0.203719  0.128802   1.0   
answer10    0.000104  0.283834  0.162836  0.317658  0.106055  0.198928   1.0   

              AU09      AU10  AU11  ...

          Confidence      AU01      AU02      AU04      AU05      AU06  AU07  \
label                                                                          
answer1          NaN       NaN       NaN       NaN       NaN       NaN   NaN   
answer2     0.018685  0.427825  0.554401  0.623122  0.314264  0.273400   1.0   
answer3     0.051843  0.483874  0.526082  0.571601  0.241101  0.453696   1.0   
answer4          NaN       NaN       NaN       NaN       NaN       NaN   NaN   
answer5     0.065697  0.383506  0.442336  0.659371  0.339540  0.484200   1.0   
answer6          NaN       NaN       NaN       NaN       NaN       NaN   NaN   
answer7          NaN       NaN       NaN       NaN       NaN       NaN   NaN   
answer8     0.086636  0.439000  0.591283  0.626923  0.397121  0.521460   1.0   
answer9     0.017553  0.319238  0.517286  0.633952  0.257433  0.480405   1.0   
answer10    0.044924  0.347278  0.422841  0.566764  0.211905  0.553408   1.0   

              AU09      AU10  AU11  ...

          Confidence      AU01      AU02      AU04      AU05      AU06  AU07  \
label                                                                          
answer1     0.002719  0.360654  0.465614  0.523606  0.279879  0.204858   1.0   
answer2     0.002235  0.461798  0.445201  0.519427  0.266131  0.231133   1.0   
answer3     0.005324  0.421683  0.528017  0.498970  0.321364  0.516543   1.0   
answer4     0.004595  0.532072  0.674497  0.373051  0.396057  0.075099   0.0   
answer5     0.004819  0.478561  0.595295  0.660378  0.237194  0.241530   1.0   
answer6     0.002455  0.577026  0.622163  0.449744  0.324986  0.151484   1.0   
answer7     0.001795  0.486030  0.530900  0.798713  0.289085  0.358769   1.0   
answer8     0.002269  0.551851  0.583282  0.569516  0.287641  0.388379   1.0   
answer9     0.001010  0.141046  0.234493  0.326101  0.308123  0.093353   1.0   
answer10    0.001783  0.447920  0.469841  0.544303  0.243798  0.187117   1.0   

              AU09      AU10  AU11  ...

In [25]:
# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []
correct_labels_list = []  # 正解ラベルを保存するリスト

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # XGBoostモデルの初期化と訓練
    model = xgb.XGBClassifier(eval_metric='logloss')
    model.fit(X_train_combined, y_train_combined)
    
    # 予測
    y_pred = model.predict(X_test)
    
    # 精度計算
    correct_predictions = (y_pred == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100  # 予測率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    correct_labels_list.append(y_test)  # 正解ラベルも保存
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.iloc[0]}")  # 最初の正解ラベルのみ表示
    print(f"予測ラベル: {y_pred}")  # 予測ラベル全体を表示

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = 0
for (file_name, acc), y_test in zip(accuracy_list, correct_labels_list):
    if 1 in y_test.values and acc >= 50:  # 正解ラベルが1を含んでいる場合
        count_above_50_for_label_1 += 1

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
ファイル名: filtered_ID30.csv
予測率: 100.00%
正解ラベル: 0
予測ラベル: [0 0 0 0 0 0 0 0 0 0]

===== テストデータ: ファイル 2/31 =====
ファイル名: filtered_ID31.csv
予測率: 90.00%
正解ラベル: 0
予測ラベル: [0 0 0 0 0 0 1 0 0 0]

===== テストデータ: ファイル 3/31 =====
ファイル名: filtered_ID32.csv
予測率: 20.00%
正解ラベル: 1
予測ラベル: [0 0 0 0 0 1 1 0 0 0]

===== テストデータ: ファイル 4/31 =====
ファイル名: filtered_ID33.csv
予測率: 80.00%
正解ラベル: 0
予測ラベル: [0 0 0 0 1 0 0 1 0 0]

===== テストデータ: ファイル 5/31 =====
ファイル名: filtered_ID34.csv
予測率: 0.00%
正解ラベル: 1
予測ラベル: [0 0 0 0 0 0 0 0]

===== テストデータ: ファイル 6/31 =====
ファイル名: filtered_ID35.csv
予測率: 100.00%
正解ラベル: 0
予測ラベル: [0 0 0 0 0 0 0 0 0]

===== テストデータ: ファイル 7/31 =====
ファイル名: filtered_ID36.csv
予測率: 60.00%
正解ラベル: 0
予測ラベル: [0 0 1 1 1 0 0 0 1 0]

===== テストデータ: ファイル 8/31 =====
ファイル名: filtered_ID37.csv
予測率: 90.00%
正解ラベル: 0
予測ラベル: [0 0 0 1 0 0 0 0 0 0]

===== テストデータ: ファイル 9/31 =====
ファイル名: filtered_ID38.csv
予測率: 100.00%
正解ラベル: 0
予測ラベル: [0 0 0 0 0 0 0 0 0 0]

===== テストデータ: ファイル 10/31 =====
ファイル名: filtered_ID

In [6]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []
correct_labels_list = []  # 正解ラベルを保存するリスト

# グリッドサーチ用のパラメータ設定
param_grid = {
    'max_depth': [3, 5, 7],           # 木の最大深さ
    'learning_rate': [0.1, 0.01],     # 学習率
    'n_estimators': [50, 100, 200],   # 木の数
    'subsample': [0.8, 1.0],          # サブサンプリング比率
    'colsample_bytree': [0.8, 1.0],   # 特徴量サンプリング比率
}

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    test_df_cleaned = test_df_cleaned.dropna(axis=0)  # NaNを含む行を削除
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            train_df_cleaned = train_df_cleaned.dropna(axis=0)  # NaNを含む行を削除
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # XGBoostモデルをグリッドサーチでチューニング
    xgb_model = xgb.XGBClassifier(eval_metric='logloss')
    grid_search = GridSearchCV(
        estimator=xgb_model,
        param_grid=param_grid,
        scoring='accuracy',   # 評価指標は正答率
        cv=3,                 # 3分割交差検証
        n_jobs=-1,            # 並列処理
        verbose=1             # 進捗状況を表示
    )
    
    # グリッドサーチ実行
    grid_search.fit(X_train_combined, y_train_combined)
    best_model = grid_search.best_estimator_  # 最良モデルを取得
    
    # 予測
    y_pred = best_model.predict(X_test)
    
    # 精度計算
    correct_predictions = (y_pred == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100  # 予測率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    correct_labels_list.append(y_test)  # 正解ラベルも保存
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"最適パラメータ: {grid_search.best_params_}")
    print(f"正解ラベル: {y_test.iloc[0]}")
    print(f"予測ラベル: {y_pred}")

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = 0
for (file_name, acc), y_test in zip(accuracy_list, correct_labels_list):
    if 1 in y_test.values and acc >= 50:  # 正解ラベルが1を含んでいる場合
        count_above_50_for_label_1 += 1

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
Fitting 3 folds for each of 72 candidates, totalling 216 fits
ファイル名: filtered_ID30.csv
予測率: 100.00%
最適パラメータ: {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 50, 'subsample': 0.8}
正解ラベル: 0
予測ラベル: [0 0 0 0 0 0 0 0 0 0]

===== テストデータ: ファイル 2/31 =====
Fitting 3 folds for each of 72 candidates, totalling 216 fits
ファイル名: filtered_ID31.csv
予測率: 70.00%
最適パラメータ: {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.8}
正解ラベル: 0
予測ラベル: [0 1 0 1 0 0 1 0 0 0]

===== テストデータ: ファイル 3/31 =====
Fitting 3 folds for each of 72 candidates, totalling 216 fits
ファイル名: filtered_ID32.csv
予測率: 0.00%
最適パラメータ: {'colsample_bytree': 1.0, 'learning_rate': 0.01, 'max_depth': 7, 'n_estimators': 50, 'subsample': 1.0}
正解ラベル: 1
予測ラベル: [0 0 0 0 0 0 0 0 0 0]

===== テストデータ: ファイル 4/31 =====
Fitting 3 folds for each of 72 candidates, totalling 216 fits
ファイル名: filtered_ID33.csv
予測率: 70.00%
最適パラメータ: {'colsample_bytree': 1

ファイル名: filtered_ID73.csv
予測率: 88.89%
最適パラメータ: {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 50, 'subsample': 1.0}
正解ラベル: 0
予測ラベル: [0 0 0 0 0 0 0 1 0]

===== 全ファイルごとの予測率 =====
ファイル名: filtered_ID30.csv, 予測率: 100.00%
ファイル名: filtered_ID31.csv, 予測率: 70.00%
ファイル名: filtered_ID32.csv, 予測率: 0.00%
ファイル名: filtered_ID33.csv, 予測率: 70.00%
ファイル名: filtered_ID34.csv, 予測率: 0.00%
ファイル名: filtered_ID35.csv, 予測率: 100.00%
ファイル名: filtered_ID36.csv, 予測率: 70.00%
ファイル名: filtered_ID37.csv, 予測率: 90.00%
ファイル名: filtered_ID38.csv, 予測率: 100.00%
ファイル名: filtered_ID39.csv, 予測率: 100.00%
ファイル名: filtered_ID40.csv, 予測率: 100.00%
ファイル名: filtered_ID41.csv, 予測率: 80.00%
ファイル名: filtered_ID42.csv, 予測率: 100.00%
ファイル名: filtered_ID43.csv, 予測率: 0.00%
ファイル名: filtered_ID44.csv, 予測率: 0.00%
ファイル名: filtered_ID45.csv, 予測率: 62.50%
ファイル名: filtered_ID46.csv, 予測率: 10.00%
ファイル名: filtered_ID47.csv, 予測率: 66.67%
ファイル名: filtered_ID48.csv, 予測率: 70.00%
ファイル名: filtered_ID49.csv, 予測率: 66.67%
ファイル名: filtered_ID50.csv, 予測率

## xgboost（先頭10フレームの最大値と最小値の差分）

In [17]:
# 処理するフォルダのパス
folder_path = './output_filtered_questions'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `answer1`から`answer10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^answer(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"answer{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の10フレームを取得
    first_10_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(10))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )

    # ラベルごとに最大値を計算
    max_values_per_label = (
        first_10_frames_per_label.groupby("label").max(numeric_only=True)
    )

    # ラベルごとに最小値を計算
    min_values_per_label = (
        first_10_frames_per_label.groupby("label").min(numeric_only=True)
    )

    # 最大値と最小値の差分を計算
    range_values_per_label = max_values_per_label - min_values_per_label

    
    # 結果をリストに保存
    result_list.append((file_name, range_values_per_label))

    # Yを追加
    range_values_per_label["Y"] = labels[i]
    i = i + 1
    
# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30.csv:
          Confidence      AU01      AU02      AU04      AU05      AU06  AU07  \
label                                                                          
answer1     0.003534  0.210704  0.243156  0.281124  0.197569  0.120037   1.0   
answer2     0.003334  0.221618  0.229155  0.302576  0.196970  0.143961   1.0   
answer3     0.004746  0.340190  0.485027  0.354739  0.217492  0.120096   1.0   
answer4     0.004306  0.226259  0.251805  0.227426  0.292127  0.123546   1.0   
answer5     0.005634  0.230452  0.271130  0.284695  0.303500  0.133120   1.0   
answer6     0.004848  0.180600  0.392590  0.277911  0.269463  0.070650   0.0   
answer7     0.004615  0.268238  0.366938  0.329792  0.477615  0.133785   1.0   
answer8     0.004237  0.182690  0.215836  0.400878  0.274844  0.101990   1.0   
answer9     0.003143  0.343358  0.277812  0.288465  0.212358  0.072390   0.0   
answer10    0.005723  0.234590  0.243214  0.361977  0.267432  0.064111   1.0   

        

          Confidence      AU01      AU02      AU04      AU05      AU06  AU07  \
label                                                                          
answer1     0.000428  0.142497  0.136217  0.120336  0.242856  0.025954   1.0   
answer2     0.001026  0.079904  0.116326  0.269431  0.080023  0.030745   0.0   
answer3     0.000331  0.098087  0.155187  0.215776  0.121682  0.057604   0.0   
answer4     0.000702  0.049766  0.150075  0.273963  0.106617  0.031923   0.0   
answer5          NaN       NaN       NaN       NaN       NaN       NaN   NaN   
answer6     0.000572  0.116929  0.079069  0.248890  0.089239  0.078845   1.0   
answer7     0.000869  0.475750  0.506268  0.460405  0.093547  0.073863   1.0   
answer8     0.000941  0.085370  0.231397  0.421611  0.195041  0.052191   1.0   
answer9     0.000627  0.137781  0.240606  0.273665  0.226873  0.118843   1.0   
answer10    0.000627  0.094151  0.088129  0.272547  0.086448  0.079216   1.0   

              AU09      AU10  AU11  ...

In [27]:
# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []
correct_labels_list = []  # 正解ラベルを保存するリスト

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # XGBoostモデルの初期化と訓練
    model = xgb.XGBClassifier(eval_metric='logloss')
    model.fit(X_train_combined, y_train_combined)
    
    # 予測
    y_pred = model.predict(X_test)
    
    # 精度計算
    correct_predictions = (y_pred == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100  # 予測率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    correct_labels_list.append(y_test)  # 正解ラベルも保存
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.iloc[0]}")  # 最初の正解ラベルのみ表示
    print(f"予測ラベル: {y_pred}")  # 予測ラベル全体を表示

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = 0
for (file_name, acc), y_test in zip(accuracy_list, correct_labels_list):
    if 1 in y_test.values and acc >= 50:  # 正解ラベルが1を含んでいる場合
        count_above_50_for_label_1 += 1

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
ファイル名: filtered_ID30.csv
予測率: 80.00%
正解ラベル: 0
予測ラベル: [0 0 1 0 0 0 0 1 0 0]

===== テストデータ: ファイル 2/31 =====
ファイル名: filtered_ID31.csv
予測率: 60.00%
正解ラベル: 0
予測ラベル: [0 0 1 1 0 1 0 0 0 1]

===== テストデータ: ファイル 3/31 =====
ファイル名: filtered_ID32.csv
予測率: 10.00%
正解ラベル: 1
予測ラベル: [0 0 0 0 0 0 0 0 0 1]

===== テストデータ: ファイル 4/31 =====
ファイル名: filtered_ID33.csv
予測率: 70.00%
正解ラベル: 0
予測ラベル: [1 0 1 0 1 0 0 0 0 0]

===== テストデータ: ファイル 5/31 =====
ファイル名: filtered_ID34.csv
予測率: 12.50%
正解ラベル: 1
予測ラベル: [0 0 0 1 0 0 0 0]

===== テストデータ: ファイル 6/31 =====
ファイル名: filtered_ID35.csv
予測率: 66.67%
正解ラベル: 0
予測ラベル: [0 1 0 0 1 1 0 0 0]

===== テストデータ: ファイル 7/31 =====
ファイル名: filtered_ID36.csv
予測率: 90.00%
正解ラベル: 0
予測ラベル: [0 0 0 0 1 0 0 0 0 0]

===== テストデータ: ファイル 8/31 =====
ファイル名: filtered_ID37.csv
予測率: 80.00%
正解ラベル: 0
予測ラベル: [0 0 1 0 0 0 0 0 0 1]

===== テストデータ: ファイル 9/31 =====
ファイル名: filtered_ID38.csv
予測率: 100.00%
正解ラベル: 0
予測ラベル: [0 0 0 0 0 0 0 0 0 0]

===== テストデータ: ファイル 10/31 =====
ファイル名: filtered_ID3

In [18]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []
correct_labels_list = []  # 正解ラベルを保存するリスト

# グリッドサーチ用のパラメータ設定
param_grid = {
    'max_depth': [3, 5, 7],           # 木の最大深さ
    'learning_rate': [0.1, 0.01],     # 学習率
    'n_estimators': [50, 100, 200],   # 木の数
    'subsample': [0.8, 1.0],          # サブサンプリング比率
    'colsample_bytree': [0.8, 1.0],   # 特徴量サンプリング比率
}

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    test_df_cleaned = test_df_cleaned.dropna(axis=0)  # NaNを含む行を削除
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            train_df_cleaned = train_df_cleaned.dropna(axis=0)  # NaNを含む行を削除
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # XGBoostモデルをグリッドサーチでチューニング
    xgb_model = xgb.XGBClassifier(eval_metric='logloss')
    grid_search = GridSearchCV(
        estimator=xgb_model,
        param_grid=param_grid,
        scoring='accuracy',   # 評価指標は正答率
        cv=3,                 # 3分割交差検証
        n_jobs=-1,            # 並列処理
        verbose=1             # 進捗状況を表示
    )
    
    # グリッドサーチ実行
    grid_search.fit(X_train_combined, y_train_combined)
    best_model = grid_search.best_estimator_  # 最良モデルを取得
    
    # 予測
    y_pred = best_model.predict(X_test)
    
    # 精度計算
    correct_predictions = (y_pred == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100  # 予測率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    correct_labels_list.append(y_test)  # 正解ラベルも保存
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"最適パラメータ: {grid_search.best_params_}")
    print(f"正解ラベル: {y_test.iloc[0]}")
    print(f"予測ラベル: {y_pred}")

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = 0
for (file_name, acc), y_test in zip(accuracy_list, correct_labels_list):
    if 1 in y_test.values and acc >= 50:  # 正解ラベルが1を含んでいる場合
        count_above_50_for_label_1 += 1

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
Fitting 3 folds for each of 72 candidates, totalling 216 fits
ファイル名: filtered_ID30.csv
予測率: 80.00%
最適パラメータ: {'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 200, 'subsample': 1.0}
正解ラベル: 0
予測ラベル: [0 0 1 0 0 0 0 1 0 0]

===== テストデータ: ファイル 2/31 =====
Fitting 3 folds for each of 72 candidates, totalling 216 fits
ファイル名: filtered_ID31.csv
予測率: 100.00%
最適パラメータ: {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100, 'subsample': 1.0}
正解ラベル: 0
予測ラベル: [0 0 0 0 0 0 0 0 0 0]

===== テストデータ: ファイル 3/31 =====
Fitting 3 folds for each of 72 candidates, totalling 216 fits
ファイル名: filtered_ID32.csv
予測率: 10.00%
最適パラメータ: {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 200, 'subsample': 1.0}
正解ラベル: 1
予測ラベル: [0 0 0 0 0 0 1 0 0 0]

===== テストデータ: ファイル 4/31 =====
Fitting 3 folds for each of 72 candidates, totalling 216 fits
ファイル名: filtered_ID33.csv
予測率: 70.00%
最適パラメータ: {'colsample_bytree':

ファイル名: filtered_ID73.csv
予測率: 88.89%
最適パラメータ: {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 50, 'subsample': 0.8}
正解ラベル: 0
予測ラベル: [0 0 0 0 0 0 0 0 1]

===== 全ファイルごとの予測率 =====
ファイル名: filtered_ID30.csv, 予測率: 80.00%
ファイル名: filtered_ID31.csv, 予測率: 100.00%
ファイル名: filtered_ID32.csv, 予測率: 10.00%
ファイル名: filtered_ID33.csv, 予測率: 70.00%
ファイル名: filtered_ID34.csv, 予測率: 12.50%
ファイル名: filtered_ID35.csv, 予測率: 66.67%
ファイル名: filtered_ID36.csv, 予測率: 80.00%
ファイル名: filtered_ID37.csv, 予測率: 70.00%
ファイル名: filtered_ID38.csv, 予測率: 100.00%
ファイル名: filtered_ID39.csv, 予測率: 100.00%
ファイル名: filtered_ID40.csv, 予測率: 100.00%
ファイル名: filtered_ID41.csv, 予測率: 100.00%
ファイル名: filtered_ID42.csv, 予測率: 100.00%
ファイル名: filtered_ID43.csv, 予測率: 0.00%
ファイル名: filtered_ID44.csv, 予測率: 0.00%
ファイル名: filtered_ID45.csv, 予測率: 100.00%
ファイル名: filtered_ID46.csv, 予測率: 0.00%
ファイル名: filtered_ID47.csv, 予測率: 66.67%
ファイル名: filtered_ID48.csv, 予測率: 40.00%
ファイル名: filtered_ID49.csv, 予測率: 33.33%
ファイル名: filtered_ID50.csv, 